In [2]:
import os
import requests
from datetime import datetime
import pandas as pd
from urllib.parse import urlparse

In [3]:
class NewsDataProcessor:
    def __init__(self):
        self.API_KEY = "7541759cd303430f8423506f0172cf60"
        self.ENDPOINT_URL = "https://newsapi.org/v2/everything"

    def get_articles(self):
        """
        Fetch articles from News API and process them.
        Returns a DataFrame with articles.
        """
        today = datetime.now()

        # Set the start of the month
        start_of_month = datetime(today.year, today.month, 1)

        # Convert dates to string format for the API request
        today_str = today.strftime("%Y-%m-%d")
        start_of_month_str = start_of_month.strftime("%Y-%m-%d")

        # Parameters for the request
        params = {
            "q": "climate change OR global warming OR climate crisis OR sustainability OR renewable energy",
            "from": start_of_month_str,
            "to": today_str,
            "sortBy": "popularity",
            "apiKey": self.API_KEY,
        }

        response = requests.get(self.ENDPOINT_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            articles = data["articles"]
            print(
                "First Article:", articles[0]
            )  # This will print out the first article's details.

        else:
            raise Exception(
                f"Request failed with status code {response.status_code}"
            )

        # Convert the list of dictionaries into a DataFrame
        df_articles = pd.DataFrame(articles)

        # Apply the function to the source column
        df_articles["source"] = df_articles["source"].apply(
            self.ensure_dict_format
        )
        print(df_articles.columns)

        urls = df_articles["url"].tolist()
        print("Captured URLs:", urls)


    @staticmethod
    def ensure_dict_format(entry):
        """
        Ensure the entry is in dictionary format.
        If it's a string, converts it to a dictionary.
        """
        if isinstance(entry, dict):
            return entry
        elif isinstance(entry, str):
            return {"id": None, "name": entry}
        else:
            raise TypeError(
                f"Unexpected type {type(entry)} encountered in the source column"
            )

    @staticmethod
    def generate_domain_representations(source_dict):
        name = source_dict[
            "name"
        ]  # Extract the name from the source dictionary
        representations = set()

        # Convert to lowercase and remove spaces
        base_name = name.lower().replace(" ", "")

        # If it doesn't have a '.' we'll assume it's a proper name and not a domain
        if "." not in base_name:
            representations.add(base_name + ".com")
            representations.add(base_name + ".org")
            representations.add(base_name + ".net")
            representations.add("www." + base_name + ".com")
        else:
            representations.add(base_name)

            # Remove www.
            if base_name.startswith("www."):
                representations.add(base_name[4:])

            # Change .com to .co.uk and vice versa
            if base_name.endswith(".com"):
                representations.add(base_name[:-4] + ".co.uk")
            elif base_name.endswith(".co.uk"):
                representations.add(base_name[:-6] + ".com")

        return representations

    @staticmethod
    def extract_domain(url):
        return urlparse(url).netloc

    DOMAINS_TO_CHECK = {
        "ABC News": ["abcnews.go.com"],
        "Bezosearthfund.org": ["bezosearthfund.org"],
        "Boing Boing": ["boingboing.net"],
        "Business Insider": ["businessinsider.com"],
        "Colorado Public Radio": ["cpr.org"],
        "Engadget": ["engadget.com"],
        "Hubspot.com": ["hubspot.com"],
        "HuffPost": ["huffpost.com", "huffingtonpost.com"],
        "NPR": ["npr.org"],
        "Noahpinion.blog": ["noahpinion.blog"],
        "PBS": ["pbs.org"],
        "Politicopro.com": ["politicopro.com"],
        "ReadWrite": ["readwrite.com"],
        "Sanjoseinside.com": ["sanjoseinside.com"],
        "Slashdot.org": ["slashdot.org"],
        "Slaynews.com": ["slaynews.com"],
        "The Atlantic": ["theatlantic.com"],
        "The Guardian": ["theguardian.com", "guardian.co.uk"],
        "The New Yorker": ["newyorker.com"],
        "The Next Web": ["thenextweb.com"],
        "The Verge": ["theverge.com"],
        "The Washington Post": ["washingtonpost.com"],
        "Time": ["time.com"],
        "VentureBeat": ["venturebeat.com"],
        "WOODTV.com": ["woodtv.com"],
        "Yahoo Entertainment": ["yahoo.com", "entertainment.yahoo.com"],
    }

    def check_domains(
        self,
        df_articles,
        high_quality_domains,
        generate_domain_representations_func,
    ):
        found_domains = []
        not_found_domains = []

        for index, row in df_articles.iterrows():
            domain = row["domain"]
            source_dict = row["source"]
            possible_representations = self.DOMAINS_TO_CHECK.get(
                source_dict["name"], []
            )

            if not possible_representations:
                possible_representations = (
                    generate_domain_representations_func(source_dict)
                )

            if any(
                rep in high_quality_domains for rep in possible_representations
            ):
                found_domains.append(domain)
            else:
                not_found_domains.append(domain)
                df_articles.drop(index, inplace=True)

        return found_domains, not_found_domains, df_articles


In [4]:
API_KEY = "7541759cd303430f8423506f0172cf60"
ENDPOINT_URL = "https://newsapi.org/v2/everything"
today = datetime.now()
start_of_month = datetime(today.year, today.month, 1)
today_str = today.strftime("%Y-%m-%d")
start_of_month_str = start_of_month.strftime("%Y-%m-%d")

params = {
    "q": "climate change OR global warming OR climate crisis OR sustainability OR renewable energy",
    "from": start_of_month_str,
    "to": today_str,
    "sortBy": "popularity",
    "apiKey": API_KEY,
}

response = requests.get(ENDPOINT_URL, params=params)

if response.status_code == 200:
    data = response.json()
    articles = data["articles"]
    print(
        "First Article:", articles[0]
    )  # This will print out the first article's details.

else:
    raise Exception(
        f"Request failed with status code {response.status_code}"
    )



First Article: {'source': {'id': None, 'name': 'Lifehacker.com'}, 'author': 'Jeff Somers', 'title': "Here's What Would Happen If You Didn't Heat Your House All Winter", 'description': 'Everything is getting more expensive these days, and the fuel that heats your home is no exception. Whether you burn oil or gas or have electric heating, it’s going to be more expensive than ever to heat your home during the coming cold weather.Read more...', 'url': 'https://lifehacker.com/heres-what-would-happen-if-you-didnt-heat-your-house-al-1850902659', 'urlToImage': 'https://i.kinja-img.com/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/c36264f2dee1ec645c6959efd2ce3870.jpg', 'publishedAt': '2023-10-05T16:00:00Z', 'content': 'Everything is getting more expensive these days, and the fuel that heats your home is no exception. Whether you burn oil or gas or have electric heating, its going to be more expensive than ever to h… [+4258 chars]'}


In [5]:
articles[4].keys()

dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])

In [6]:
# Convert the list of dictionaries into a DataFrame
df_articles = pd.DataFrame(articles)

In [34]:
params = {
    # "q": "climate change OR global warming OR climate crisis OR sustainability OR renewable energy",
    "domains": domains_str,
    "from": today.isoformat(),
    "to": today.isoformat(),
    "sortBy": "popularity",
    "apiKey": API_KEY,
}
response = requests.get(ENDPOINT_URL, params=params)

In [35]:
len(domains_str)

76626

In [38]:
response.status_code

414

In [7]:
df_articles

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Lifehacker.com'}",Jeff Somers,Here's What Would Happen If You Didn't Heat Yo...,Everything is getting more expensive these day...,https://lifehacker.com/heres-what-would-happen...,"https://i.kinja-img.com/image/upload/c_fill,f_...",2023-10-05T16:00:00Z,Everything is getting more expensive these day...
1,"{'id': 'wired', 'name': 'Wired'}",David Gilbert,The 15-Minute City Conspiracy Theory Goes Main...,The fringe idea that cities built for biking a...,https://www.wired.com/story/15-minute-city-con...,https://media.wired.com/photos/651ae0793cfe742...,2023-10-02T15:29:34Z,"Back in February, when a Conservative lawmaker..."
2,"{'id': 'the-verge', 'name': 'The Verge'}",Justine Calma,New report casts doubt on Apple’s first ‘carbo...,"Apple says its new watches are carbon neutral,...",https://www.theverge.com/2023/10/3/23901236/ap...,https://cdn.vox-cdn.com/thumbor/R7BxmMt5_jpNSP...,2023-10-03T15:52:29Z,New report casts doubt on Apples first carbon ...
3,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Climate change: Pope Francis warns world 'may ...,Pope Francis criticises attempts to deny or gl...,https://www.bbc.co.uk/news/world-67005362,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-10-04T12:36:57Z,"Pope Francis has warned the world is ""collapsi..."
4,"{'id': None, 'name': 'NPR'}",Quil Lawrence,The military is turning to microgrids to fight...,The military is among the largest buyers of in...,https://www.npr.org/2023/10/02/1201838599/mili...,https://media.npr.org/assets/img/2023/09/28/an...,2023-10-02T16:00:48Z,Exterior of MCAS Miramar microgrid rooms in Sa...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Nakedcapitalism.com'}",Yves Smith,Links 10/6/2023,Our pugnacious daily links: rice straw critter...,https://www.nakedcapitalism.com/2023/10/links-...,https://www.nakedcapitalism.com/wp-content/upl...,2023-10-06T10:55:00Z,Giant Rice Straw Sculptures of Animals Take Ov...
96,"{'id': None, 'name': 'Nakedcapitalism.com'}",Yves Smith,BRICS: An Anti-Imperialist Fantasy and Sub-Imp...,A no-holds-barred critique of BRICS as mainly ...,https://www.nakedcapitalism.com/2023/10/brics-...,https://theanalysis.news/wp-content/uploads/20...,2023-10-04T05:00:35Z,"Hi, Im Talia Baroncelli, and youre watching th..."
97,"{'id': None, 'name': 'Nakedcapitalism.com'}",Conor Gallagher,Links 10/1/2023,None,https://www.nakedcapitalism.com/2023/10/links-...,https://www.nakedcapitalism.com/wp-content/upl...,2023-10-01T10:55:36Z,"One Million Years Ago, Our Human Relatives May..."
98,"{'id': None, 'name': 'Psychology Today'}",Marc Bekoff Ph.D.,"Dogs and Cats Can Be Healthy, Happy Vegans, Re...",Two new comprehensive studies show that plant-...,https://www.psychologytoday.com/intl/blog/anim...,https://cdn2.psychologytoday.com/assets/styles...,2023-10-05T13:41:45Z,"Can cats be healthy, happy vegans? Many people..."


In [8]:
from newsapi import NewsApiClient

In [9]:
# Init
newsapi = NewsApiClient(api_key=API_KEY)


In [ ]:
from datetime import timedelta
today = datetime.now()
start_date=  today - timedelta(days=2)
end_date = today

In [10]:
all_articles = newsapi.get_everything(
    # q='climate change OR global warming OR climate crisis OR sustainability OR renewable energy',
                                      from_param='2023-10-05',
                                      to='2023-10-07',
                                      language='en',
                                      sort_by='relevancy',)

In [12]:
len(all_articles["articles"])

100

In [13]:
all_articles

{'status': 'ok',
 'totalResults': 1605,
 'articles': [{'source': {'id': 'the-verge', 'name': 'The Verge'},
   'author': 'Justine Calma',
   'title': 'New report casts doubt on Apple’s first ‘carbon neutral’ products',
   'description': 'Apple says its new watches are carbon neutral, but a new report says the company needs to share more information about its suppliers to back that claim.',
   'url': 'https://www.theverge.com/2023/10/3/23901236/apple-carbon-neutral-watch-report-supply-chain-emissions-transparency',
   'urlToImage': 'https://cdn.vox-cdn.com/thumbor/R7BxmMt5_jpNSPxP5b9t430i7O0=/0x0:2880x1620/1200x628/filters:focal(1440x810:1441x811)/cdn.vox-cdn.com/uploads/chorus_asset/file/24973547/Apple_2030_Montague_wind_farm_sunset_Oregon_230912.jpg.large_2x.jpg',
   'publishedAt': '2023-10-03T15:52:29Z',
   'content': 'New report casts doubt on Apples first carbon neutral products\r\nNew report casts doubt on Apples first carbon neutral products\r\n / Apple needs to clean up its suppl

In [15]:
domains=['dailymail.co.uk', 'rumble.com', 'rawstory.com', 'indiatimes.com', 'breitbart.com', 'theepochtimes.com', 'thegatewaypundit.com', 'zerohedge.com', 'cbn.com', 'rt.com', 'left.it', 'newsmax.com', 'news18.com', 'gnews.org', 'newsbreak.com', 'economictimes.indiatimes.com', 'thefederalist.com', 'dailywire.com', 'dailycaller.com', 'washingtontimes.com', 'stream.org', 'childrenshealthdefense.org', 'chosun.com', 'tichyseinblick.de', 'lifenews.com', 'express.co.uk', 'dailykos.com', 'townhall.com', 'lifesitenews.com', 'theblaze.com', 'valeursactuelles.com', 'telesurtv.net', 'occupydemocrats.com', 'jihadwatch.org', 'esrt.press', 'theonion.com', 'expose-news.com', 'lemediaen442.fr', 'redstate.com', 'palmerreport.com', 'politicususa.com', 'bitchute.com', 'lecourrier-du-soir.com', 'elamerican.com', 'rsbnetwork.com', 'alternet.org', 'health.news', 'amgreatness.com', 'judicialwatch.org', 'firstpost.com', 'peta.org', 'politicalflare.com', 'westernjournal.com', 'thecountersignal.com', 'bvoltaire.fr', 'greenpeace.org', 'wsws.org', 'jezebel.com', 'hannity.com', 'francesoir.fr', 'pagesix.com', 'twitchy.com', 'skynews.com.au', 'laverita.info', 'ria.ru', 'babylonbee.com', 'crooksandliars.com', 'breaking911.com', 'sabah.com.tr', 'reseauinternational.net', 'pjmedia.com', 'beckernews.com', 'bipartisanreport.com', 'sputniknews.com', 'planetes360.fr', 'ancient-origins.net', 'newsbusters.org', 'swarajyamag.com', 'americanthinker.com', 'truthout.org', 'theconservativetreehouse.com', 'democrats.org', 'lavoixdunord.fr', 'achgut.com', 'thenationalpulse.com', 'elmundotoday.com', 'report24.news', 'imolaoggi.it', 'outkick.com', 'arabnews.com', 'english.news.cn', 'wnd.com', 'byoblu.com', 'english.alarabiya.net', 'dailystar.co.uk', 'epochtimes.com', 'dailysignal.com', 'newyorker.com/humor', 'livescience.com', 'naturalnews.com', 'tass.com', 'newspunch.com', 'epochtimes.jp', 'chinadaily.com.cn', 'toddstarnes.com', 'mintpressnews.com', 'newyorker.com/humor/borowitz-report', 'protothema.gr', 'revolver.news', 'nicolaporro.it', 'policeetrealites.com', 'tgcom24.mediaset.it', 'etonline.com', 'mondoweiss.net', 'saraacarter.com', 'ripostelaique.com', 'summit.news', 'thelondoneconomic.com', 'infowars.com', 'thegrayzone.com', 'resistancerepublicaine.com', 'moveon.org', 'foxsports.com.au', 'lamontagne.fr', 'postillon.com', 'tes.com', 'notrickszone.com', 'globalresearch.ca', 'ntd.com', 'www1.cbn.com', 'rudaw.net', 'cnsnews.com', 'welovetrump.com', 'liveaction.org', 'science.news', 'alphanews.org', 'thepoliticalinsider.com', 'gellerreport.com', 'redvoicemedia.com', 'bongino.com', 'nowthisnews.com', 'hola.com', 'mises.org', 'sott.net', 'linkiesta.it', 'oann.com', 'cogiito.com', 'fox10phoenix.com', 'wonkette.com', 'waynedupree.com', 'bestnewshere.com', 'answersingenesis.org', 'gop.gov', 'patriotpost.us', 'gatestoneinstitute.org', 'counterpunch.org', 'gop.com', 'qactus.fr', 'telesurenglish.net', 'americasvoice.news', 'atlantico.fr', 'azmirror.com', 'wattsupwiththat.com', 'deutschlandkurier.de', 'investors.com', 'lantidiplomatico.it', 'technocracy.news', 'ns2017.wordpress.com', 'ilprimatonazionale.it', 'nationalfile.com', 'legorafi.fr', 'theinfowar.tv', 'research.news', 'trump.news', 'icandecide.org', 'thenewamerican.com', 'epochtimes.de', 'pravda.ru', 'stopcensura.online', 'trendingpolitics.com', 'antiwar.com', 'frontpagemag.com', 'conservativewoman.co.uk', 'milanofinanza.it', 'nerdist.com', 'anguillesousroche.com', 'wochenblick.at', 'toofab.com', 'truthpress.com', 'maurizioblondet.it', 'epochtimes.fr', 'donbass-insider.com', 'esrt.online', 'onegreenplanet.org', 'wikileaks.org', 'affaritaliani.it', 'prageru.com', 'thegreggjarrett.com', 'theshovel.com.au', 'parstoday.com', 'thehighwire.com', 'becauseofthemwecan.com', 'eventiavversinews.it', 'beckershospitalreview.com', 'food.news', 'analyzingamerica.org', 'herb.co', 'secondnexus.com', 'cair.com', 'space.news', 'ladbible.com', 'mag24.cloud', 'voltairenet.org', 'quillette.com', 'meidastouch.com', 'therightscoop.com', 'pressfortruth.ca', 'thelibertydaily.com', 'fox9.com', 'de.rt.com', 'boundingintocomics.com', 'climate.news', 'hightimes.com', 'ussanews.com', 'perezhilton.com', 'tasnimnews.com', 'friatider.se', 'lalettrepatriote.com', 'aubedigitale.com', 'spectator.org', 'gear.news', 'endtimeheadlines.org', 'thehardtimes.net', 'democraticunderground.com', 'formiche.net', 'vdare.com', 'scenarieconomici.it', 'mindbodygreen.com', 'disclose.tv', 'globalist.it', 'conservativedailynews.com', 'lewrockwell.com', '100percentfedup.com', 'ukcolumn.org', 'wmtw.com', 'newstarget.com', 'forbes.it', 'amac.us', 'tg.la7.it', 'thepoke.co.uk', 'francais.rt.com', 'nowtheendbegins.com', 'beforeitsnews.com', 'moonofalabama.org', 'usatodaysun.com', 'powerlineblog.com', 'bearingarms.com', 'tpusa.com', 'basnews.com', 'tbdailynews.com', 'activistpost.com', 'propaganda.news', 'worldstarhiphop.com', 'egaliteetreconciliation.fr', 'louderwithcrowder.com', 'thenationaltelegraph.com', 'littlethings.com', 'halturnerradioshow.com', 'voxnews.info', 'bigleaguepolitics.com', 'republicandaily.com', 'compact-online.de', 'dworkinreport.com', 'resilience.org', 'vocm.com', 'stopworldcontrol.com', 'inflation.news', 'breizh-info.com', 'climatechangedispatch.com', 'nuclear.news', 'china.org.cn', 'grandeinganno.it', 'frankspeech.com', 'newswars.com', 'worldcouncilforhealth.org', 'unz.com', 'clashdaily.com', 'blabber.buzz', 'lesobservateurs.ch', 'canadafreepress.com', 'powerofpositivity.com', 'albawaba.com', 'livelovefruit.com', 'countercurrents.org', 'marieclaire.fr', 'bonginoreport.com', 'wingsoverscotland.com', 'americasfreedomfighters.com', 'billoreilly.com', 'tennesseestar.com', 'lesmoutonsenrages.fr', 'vigilantcitizen.com', 'donaldjtrump.com', 'politikstube.com', 'sudradio.fr', 'wisconsinexaminer.com', 'libertarianinstitute.org', 'npr.news', 'sohu.com', 'flagandcross.com', 'greatgameindia.com', 'aflds.org', 'almanar.com.lb', 'climatedepot.com', 'covid19criticalcare.com', 'canadiandimension.com', 'pandata.org', 'thedailymash.co.uk', 'blackagendareport.com', 'rightwingwatch.org', 'medias-presse.info', 'databaseitalia.it', 'prevention.com', 'lesalonbeige.fr', 'ilmeteo.it', 'conservativeundergroundnews.com', 'm.epochtimes.fr', 'weaselzippers.us', 'biased.news', 'nutritionfacts.org', 'dailysnark.com', 'natural.news', 'gospanews.net', 'paine.tv', 'chaser.com.au', 'gotquestions.org', 'dreddymd.com', 'panorama.it', 'lasicilia.it', 'xinhuanet.com', 'davidicke.com', 'inthesetimes.com', 'churchmilitant.com', 'independentsentinel.com', 'sgtreport.com', 'srbin.info', 'clickhole.com', 'bossip.com', '21stcenturywire.com', 'worldtribune.com', 'doctors4covidethics.org', 'lelibrepenseur.org', 'lifezette.com', 'strategies.fr', 'grunge.com', 'hillsdale.edu', 'nouvelordremondial.cc', 'blacklistednews.com', 'uafreport.com', 'channel33news.com', 'citizens.news', 'biotech.news', 'headtopics.com', 'ruptly.tv', 'investmentwatchblog.com', 'thebeltwayreport.com', 'organicconsumers.org', 'vaccineimpact.com', 'chippewa.com', 'controinformazione.info', 'covid19reporter.com', 'mystateline.com', 'livestrong.com', 'fairus.org', 'apolut.net', 'brandnewtube.com', 'waterfordwhispersnews.com', 'womenarehuman.com', 'dreuz.info', 'floridaphoenix.com', 'corbettreport.com', 'thepublicdiscourse.com', 'europe-israel.org', 'memri.org', 'christianheadlines.com', 'healthychildren.org', 'thelibertyloft.com', 'qanon.it', 'wikistrike.com', 'newagebd.net', 'drrichswier.com', 'coasttocoastam.com', 'principia-scientific.com', 'thedcpatriot.com', 'patriotproject.com', 'uncut-news.ch', 'mypanhandle.com', 'nevadacurrent.com', 'infobrics.org', 'tankerenemy.com', 'kla.tv', 'uncoverdc.com', 'bonsens.info', 'kfvs12.com', 'finishtherace.com', 'nordicmonitor.com', 'english.pravda.ru', 'usawatchdog.com', 'workers.org', 'thesaker.is', 'papaboys.org', 'echelledejacob.blogspot.com', 'survivethenews.com', 'geoengineeringwatch.org', 'yorknewstimes.com', 'x22report.com', 'attitude.co.uk', 'americanactionnews.com', 'fresh.news', 'outbreak.news', 'caixinglobal.com', 'patriotpulse.net', 'conservativefiringline.com', '3aw.com.au', 'everytown.org', 'mustreadalaska.com', 'ozarksfirst.com', 'tuckercarlson.com', 'caldronpool.com', 'liberty.news', 'epochtimes.se', 'crisismagazine.com', 'takimag.com', 'wakingtimes.com', 'contropiano.org', 'veteranstoday.com', 'icr.org', 'naturalsociety.com', 'epochtimes.com.br', 'takecontrol.substack.com', 'toba60.com', 'dailytorch.com', 'theeconomiccollapseblog.com', 'smobserved.com', 'sputnik.kz', 'whowhatwhy.org', 'rumormillnews.com', 'rtde.tech', 'humansbefree.com', 'paulcraigroberts.org', 'blitzquotidiano.it', 'bdnews24.com', 'power.news', 'it.aleteia.org', 'rassegneitalia.info', 'unzensuriert.at', 'eutimes.net', 'worldtruth.tv', 'censored.news', 'rightnewswire.com', 'realrawnews.com', 'whatfinger.com', 'barenakedislam.com', 'goodsciencing.com', 'clarkcountytoday.com', 'doctoroz.com', 'thewashingtonstandard.com', 'americasfrontlinedoctors.org', 'maxim.com', 'newsvirginian.com', 'trumptrainnews.com', 'manhattan-institute.org', 'ewg.org', 'journalism.news', 'winterwatch.net', 'swprs.org', 'thefreethoughtproject.com', 'thriveglobal.com', 'comedonchisciotte.org', 'physiciansforinformedconsent.org', 'pandemic.news', 'gmwatch.org', 'americasvoice.org', 'strangesounds.org', 'chicksonright.com', 'wolfstreet.com', 'truth.news', 'thetrumpet.com', 'migrants.news', 'meforum.org', 'en.mehrnews.com', 'zenith.news', 'cowichanvalleycitizen.com', 'cei.org', 'thedesertreview.com', 'judithcurry.com', 'en.mercopress.com', 'renovatio21.com', 'claremont.org', 'iowaclimate.org', 'stateofthenation.co', 'lesakerfrancophone.fr', 'scarymommy.com', 'libertynation.com', 'luogocomune.net', 'trend-online.com', 'focusonthefamily.com', 'creation.com', 'noqreport.com', 'war.ukraine.ua', 'mountsinai.org', 'english.almanar.com.lb', 'en.protothema.gr', 'francais.cgtn.com', 'transition-news.org', 'banned.video', 'freedomworks.org', 'nexus.fr', 'cfact.org', 'heartland.org', 'voice-online.co.uk', 'tehrantimes.com', 'sonsoflibertymedia.com', 'parler.com', 'gaia.com', 'remnantnewspaper.com', 'cedars-sinai.org', 'savebritain.org', 'westonaprice.org', 'informationclearinghouse.info', 'wikispooks.com', 'warroom.org', 'centerforsecuritypolicy.org', 'thereligionofpeace.com', 'charismanews.com', 'prepareforchange.net', 'breakthrough.news', 'sciencetimes.com', 'evolutionnews.org', 'whatdoesitmean.com', 'wtvq.com', 'junkscience.com', 'ipatriot.com', 'needtoknow.news', 'newspress.com', 'informationliberation.com', 'honest.news', 'prophecy.news', 'publishedreporter.com', 'sciencefiles.org', 'stillnessinthestorm.com', 'thesuburban.com', 'conservativereview.com', 'projectveritas.com', 'stupid.news', 'desantisdaily.com', 'parkrapidsenterprise.com', 'mercola.com', 'cancer.news', 'dont-tread-on.me', 'canadaaction.ca', 'mercatornet.com', 'cookcountyrecord.com', 'dailygrail.com', 'renewedright.com', 'prophecynewswatch.com', 'factcheck.news', 'stoppingsocialism.com', 'christiantoday.com', 'deepleftfield.info', 'conspiracy.news', 'eng.majalla.com', 'populistpress.com', 'thefreedomtimes.com', 'bjreview.com', 'computing.news', 'dcdirtylaundry.com', 'freiewelt.net', 'msnbc.news', 'freespeech.org', 'dentistry.news', 'meteoweb.eu', 'iharare.com', 'geopolitics.co', 'aish.com', 'southafricatoday.net', 'medicine.news', 'survival.news', 'ae911truth.org', 'adflegal.org', 'wearechange.org', 'nationalrighttolifenews.org', 'passeportsante.net', 'forbiddenknowledgetv.net', 'thepulse.one', 'tfp.org', 'agupdate.com', 'hoax.news', 'heartlanddailynews.com', 'catholic.org', 'trunews.com', 'betootaadvocate.com', 'gospelnewsnetwork.org', 'operationdisclosureofficial.com', 'thetruthseeker.co.uk', 'c19ivermectin.com', 'geologyin.com', 'lagazettedemontpellier.fr', 'glennbeck.com', 'thelibertybeacon.com', 'carm.org', 'vidmax.com', 'amren.com', 'capitalresearch.org', 'disinformazione.it', 'mamamia.com.au', 'discoverthenetworks.org', 'freerepublic.com', 'tuttiicriminidegliimmigrati.com', 'citizensjournal.us', 'jp.sputniknews.com', 'dailyboulder.com', 'neonnettle.com', 'freedomfirstnetwork.com', 'americanlookout.com', 'sputniknews.cn', 'risk.news', 'c19early.com', 'theblacksphere.net', 'birminghamtimes.com', 'telebari.it', 'thecommonsenseshow.com', 'postcard.news', 'jbs.org', 'lequotidiendupharmacien.fr', 'australiannationalreview.com', 'dcclothesline.com', 'freedom.news', 'numero6.org', 'freedomsphoenix.com', 'michiganchronicle.com', 'henrymakow.com', 'greenpass.news', 'defiantamerica.com', 'elizabethjohnston.org', 'realclimatescience.com', 'ancient-code.com', 'exoportail.com', 'thevaccinereaction.org', 'thoughtcatalog.com', 'rense.com', 'joannenova.com.au', 'houstonpress.com', 'redrightdaily.com', 'autismspeaks.org', 'blacknews.com', 'corruption.news', 'techstartups.com', 'friendsofscience.org', 'nogeoingegneria.com', 'au.sports.yahoo.com', 'freewestmedia.com', 'lesmoutonsrebelles.com', 'themillenniumreport.com', 'acpeds.org', 'endtime.com', 'americafirstpolicy.com', 'gregpalast.com', 'medicalkidnap.com', 'newshourfirst.com', 'dailyheadlines.com', 'truthforhealth.org', 'newsbiscuit.com', 'guarda.link', 'chaos.news', 'epochtimes.cz', 'thescoop.us', 'anna-news.info', 'standforhealthfreedom.com', 'unshackledminds.com', 'fromthetrenchesworldreport.com', 'conservapedia.com', 'theduran.com', 'jacobinmag.com', 'genesiustimes.com', 'breakpoint.org', 'newswithviews.com', 'theunionjournal.com', 'dailysquib.co.uk', 'hotsr.com', 'america.cgtn.com', 'freeworldnews.tv', 'violence.news', 'kcna.kp', 'sadefenza.blogspot.com', 'pandora.tv', 'ilfriuli.it', 'plandemicseries.com', 'beto.news', 'buffalochronicle.com', 'thehornnews.com', 'patrioticviralnews.com', 'disaster.news', 'syrianews.cc', 'trofire.com', 'latitudes.org', 'ghost.report', 'mittdolcino.com', 'shelbycountyreporter.com', 'ilfarosulmondo.it', 'clinton.news', 'redice.tv', 'naturalblaze.com', 'nomorefakenews.com', 'santeplusmag.com', 'dailyexpose.uk', 'heart.news', 'sakeritalia.it', 'hackthematrix.it', 'allnewspipeline.com', 'chemistry.news', 'fukushima.news', 'hereistheevidence.com', 'ho1.us', 'stalkerzone.org', 'wakeup-world.com', 'libertybugle.com', 'nationandstate.com', 'christianophobie.fr', 'ratical.org', 'breakingchristiannews.com', 'censorship.news', 'www.xinhuanet.com', 'godlikeproductions.com', 'brightside.me', 'brujitafr.fr', 'micdroppolitics.com', 'addiction.news', 'draxe.com', 'freedomplatform.tv', 'frontpagelive.com', 'greenmedinfo.com', 'shtfplan.com', 'chicagodefender.com', 'dailypoliticalnewswire.com', 'wikiislam.net', 'legitgov.org', 'ahwatukee.com', 'larouchepac.com', 'watchdogreport.org', 'libertywire.net', 'kamalaharris.news', 'gishgallop.com', 'endoftheamericandream.com', 'drsircus.com', 'epa.news', 'fda.news', 'realnewsrightnow.com', 'restoredrepublic.co', 'vaccines.news', 'neopresse.com', 'usasupreme.com', 'centrometeoitaliano.it', 'christianscience.com', 'realjewnews.com', 'metapedia.org', '100giornidaleoni.it', 'curioctopus.fr', 'sputnik-abkhazia.ru', 'britainfirst.org', 'stormer-daily.rw', 'americangulag.org', 'goop.com', 'cc.org', 'electroverse.net', 'greanvillepost.com', 'abovetopsecret.com', 'aapsonline.org', 'derehamtimes.co.uk', 'overdose.news', 'digifection.com', 'aim4truth.org', 'mrc.org', 'africanexponent.com', 'pianetablunews.it', 'sickchirpse.com', 'avoiceformen.com', 'front.moveon.org', 'brighteon.com', 'twisted.news', 'georgiarecord.com', 'emirates247.com', 'lucianne.com', 'conservativehq.org', 'fawkes-news.com', 'msg.news', 'americanpatriotdaily.com', 'metals.news', 'wvrecord.com', 'thecovidworld.com', 'metronews.it', 'thesmokersclub.com', 'everydayconcerned.net', 'mind.news', 'worldviewweekend.com', 'medicalmedium.com', 'muslimnews.co.uk', 'percolately.com', 'rtde.life', 'lacombeonline.com', 'returntonow.net', 'truth11.com', 'blacklivesmatter.com', 'genzconservative.com', 'thepeoplescube.com', 'mundo.sputniknews.com', 'silverdoctors.com', 'afa.net', 'dailysurge.com', 'aim.org', 'offgridsurvival.com', 'ageofautism.com', 'worldnewsdailyreport.com', 'energycitizens.org', 'conservativeangle.com', 'contro.tv', 'evil.news', 'faked.news', 'humansarefree.com', 'nationaltimesaustralia.com', 'silenziefalsita.it', 'english.cctv.com', 'dailyhealthpost.com', 'splinternews.com', 'worldhealth.net', 'biologos.org', 'collapse.news', 'dailypatriotreport.com', 'ilsimplicissimus2.com', 'discovery.org', 'dottoremaeveroche.it', 'mikehuckabee.com', 'oumma.com', 'texarkanagazette.com', 'theorganicprepper.com', 'frc.org', 'sonsof1776.com', 'suffolkgazette.com', 'drjockers.com', 'joebiden.news', 'teleambiente.it', 'thebluestateconservative.com', 'therightstuff.biz', 'conservativefreepress.com', 'populist.press', 'suppressed.news', 'whistleblower.news', 'thrivemovement.com', 'moonbattery.com', 'illegalaliencrimereport.com', 'cryptogon.com', 'theirishsentinel.com', 'oncology.news', 'seeds.news', 'nationalvanguard.org', 'big5.sputniknews.cn', 'dailyleader.com', 'electricity.news', 'livenewsnow.com', 'psychiatry.news', 'sputnik.by', 'naturallysavvy.com', 'ilsapereepotere2.blogspot.com', 'russia-insider.com', 'stateofthenation2012.com', 'steverotter.com', 'theminnesotasun.com', 'blackgenocide.org', 'newsready.com', 'centralfloridapost.com', 'fracking.news', 'healthimpactnews.com', 'healthnutnews.com', 'thewhiterose.uk', 'iheartintelligence.com', 'thepostemail.com', 'patriotnewsdaily.com', 'chronicle.su', 'conservativecardinal.com', 'prevention.news', 'thinkamericana.com', 'jesus-is-savior.com', 'tomwoods.com', 'afipn.com.au', 'ambassadorlove.wordpress.com', 'dementia.news', 'eminetracanada.com', 'french.xinhuanet.com', 'ntd.tv', 'projectveritasaction.com', 'themindunleashed.com', 'bluntforcetruth.com', 'inews24.it', 'droite.tv', 'lies.news', 'martiallaw.news', 'secondopianonews.it', 'newsrescue.com', 'dieunbestechlichen.com', 'japaninsides.com', 'scientific.news', 'thedailybeat.org', 'terrarealtime.blogspot.com', 'christiansfortruth.com', 'moneyandmarkets.com', 'thelightpaper.co.uk', 'protrumpnews.com', 'humortimes.com', 'funker530.com', 'thetruedefender.com', 'tierneyrealnewsnetwork.substack.com', 'greatreject.org', 'dailyexpose.co.uk', 'conoscenzealconfine.it', 'agenpress.it', 'internapoli.it', 'laprovinciadilecco.it', 'educate-yourself.org', 'msnbc.website', 'independentminute.com', 'watchdognews.org', 'americanpolicy.org', 'lovethispic.com', 'sulatestagiannilannes.blogspot.com', 'conservative-daily.com', 'mainstpress.com', 'pacificpundit.com', 'oftwominds.com', 'climatesciencenews.com', 'humanitarian.news', 'teaparty.org', 'thetruthaboutcancer.com', 'shoebat.com', 'foreverconscious.com', 'thedailybell.com', 'actualidadpanamericana.com', 'frankmag.ca', 'c19vitamind.com', 'fascism.news', 'morningjournalnews.com', 'nycaribnews.com', 'us24news.com', 'sundaysportonline.co.uk', 'fascinazione.info', 'twitter.com/radiosavana', 'investmentresearchdynamics.com', 'patrioticpost.com', '24jours.com', 'apost.com', 'blackeyepolitics.com', 'realfarmacy.com', 'whitehouse.news', 'ihr.org', 'independentsciencenews.org', 'thedailywtf.com', 'ufoholic.com', 'magapill.com', 'physics.news', 'renewamerica.com', 'rodong.rep.kp', 'nationalcenter.org', 'ticinolive.ch', 'mediamass.net', 'in5d.com', 'climatealarmism.news', 'gender.news', 'humorousmathematics.com', 'naturalnewsblogs.com', 'redrightpatriot.com', 'strategika51.org', 'theconservativebrief.com', 'truthandaction.org', 'clarionproject.org', 'wallbuilders.com', 'theeventchronicle.com', 'afinalwarning.com', 'angryasianman.com', 'freshoffthepress.org', 'ru.armeniasputnik.am', 'thetruthaboutvaccines.com', 'sustainablepulse.com', 'organicfacts.net', 'everydayfeminism.com', 'rodong.rep.kp/en', 'curiousmindmagazine.com', 'panic.news', 'secession.news', 'ihealthtube.com', 'nationalinsiders.com', 'freedomproject.com', 'amelioretasante.com', 'awarenessact.com', 'cdc.news', 'nationalenquirer.com', 'oasisana.com', 'orazero.org', 'thealternativedaily.com', 'learntherisk.org', 'nrlc.org', 'echocheck.org', 'nra.org', 'direttanews.it', 'hagmannreport.com', 'chemicalviolence.com', 'conspiracydailyupdate.com', 'djhjmedia.com', 'overlords.news', 'pollution.news', 'rtde.site', 'southfront.org', 'strategic-culture.org', 'zombie.news', 'pop.org', 'bastabugie.it', 'italiafeed.com', 'plus-saine-la-vie.com', 'rightwing.news', 'beholdisrael.org', 'roguereview.net', 'chemicals.news', 'contra24.online', 'dcbusinessdaily.com', 'heftig.de', 'infections.news', 'laminutedericardo.com', 'mag24.es', 'concernedwomen.org', 'boldprogressives.org', 'anthonyblogan.com', 'conservativerevival.com', 'drudgereport.com', 'frosinonetoday.it', 'greatamericandaily.com', 'ilroma.net', 'ingredients.news', 'lacrunadellago.net', 'madisonrecord.com', 'press.foxnews.com', 'tarantobuonasera.it', 'liveleak.com', 'nexusedizioni.it', 'thephaser.com', 'cronacaqui.it', 'hartselleenquirer.com', 'sptnkne.ws', 'truepundit.com', 'returnofkings.com', 'worldsocialism.org', 'aina.org', 'weeklyworldnews.com', 'deutsch.rt.com', 'firstamendment.news', 'healthrangerreport.com', 'persianepochtimes.com', 'soros.news', 'thebl.com', 'thecouriertimes.com', 'rightwingnews.com', 'davidwolfe.com', 'redorbit.com', 'wethepeopledaily.com', 'thedayafter2012.blogspot.com', 'asian-voice.com', 'bustatroll.org', 'newsblaze.com', 'qanon.pub', 'robots.news', 'thefloridastar.com', 'whatreallyhappened.com', 'lifespa.com', 'outragedpatriot.com', 'survivalblog.com', 'foodbabe.com', 'foreignpolicyi.org', 'presscorp.org', 'en.mediamass.net', 'bees.news', 'fl24.net', 'harvest.news', 'latinatoday.it', 'nvic.org', 'patriotjournal.org', 'redstatewatcher.com', 'rtde.website', 'starvation.news', 'therussophile.org', 'trueactivist.com', 'veteranstodayarchives.com', '911truth.org', 'threepercenternation.com', 'deconstructingconventional.com', 'freetelegraph.com', 'fruits.news', 'grocery.news', 'rtde.world', 'shareblue.com', 'socalrecord.com', 'yc.news', 'americanfreepress.net', 'thebeardedpatriot.com', 'israelislamandendtimes.com', 'foodmatters.com', 'asia-pacificresearch.com', 'resistthemainstream.org', 'sovereignman.com', 'genocide.news', 'influenza.news', 'nanotechnology.news', 'rationalground.com', 'rushlimbaugh.com', 'thefranklinnewspost.com', 'themarshallreport.wordpress.com', 'thepatriotbrief.com', 'thevaultproject.org', 'vaxxter.com', 'vernoncoleman.com', 'worldgreynews.com', 'newsinsideout.com', 'prophecytoday.com', 'patriots4truth.org', 'eluxemagazine.com', 'presscalifornia.com', 'althealthworks.com', 'anonews.co', 'camdenarknews.com', 'cbsnews.us', 'epochtimes.pl', 'flbusinessdaily.com', 'lemondejuif.info', 'riza.it', 'robotics.news', 'rtde.team', 'tr.sputniknews.com', 'wisemindhealthybody.com', 'therundownlive.com', 'thetruereporter.com', 'patribotics.blog', 'raffaelepalermonews.com', 'assis.it', 'z3news.com', 'arabic.sputniknews.com', 'bestfolkmedicine.com', 'bubble.news', 'corona-blog.net', 'emp.news', 'impfkritik.de', 'kellybroganmd.com', 'nationalsecurity.news', 'pennrecord.com', 'resist.news', 'tallasseetribune.com', 'vaccinechoicecanada.com', 'infogalactic.com', 'billionbibles.org', 'thepatriotnation.net', 'newssloth.com', 'nfrw.org', 'pianetax.wordpress.com', 'home.nra.org', 'yaliberty.org', 'connectiv.events', 'insider.foxnews.com', 'linkshare.whatfinger.com', 'mental.news', 'miamistandard.news', 'trend-news.com', 'uniteamericafirst.com', 'csglobe.com', 'sanevax.org', 'anewspost.com', 'prairiestatewire.com', 'lonesto.it', 'ecplanet.org', 'expose1933.com', 'gatesofvienna.net', 'prepperwebsite.com', '2020news.de', '79days.news', 'alternativesante.fr', 'awm.com', 'centennialstatenews.com', 'faithit.com', 'guns.news', 'healthyfoodhouse.com', 'internationalhighlife.com', 'londonnewstime.com', 'principia-scientific.org', 'qalerts.app', 'surveillance.news', 'thelosangelestribune.com', 'theresistance.video', 'vactruth.com', 'yournewswire.com', 'thenationalpatriot.com', 'understandingthethreat.com', 'breakingfirst.com', 'deplorablekel.com', 'endthefed.org', 'creationwiki.org', 'redoubtnews.com', 'defendingtherepublic.org', 'ruthinstitute.org', 'thenational.ae', 'tanker-enemy.com', 'antifa.news', 'anyavien.com', 'awakening.news', 'cancerwisdom.net', 'cantcensortruth.com', 'chietitoday.it', 'conservativedailypost.com', 'created4health.org', 'extinction.news', 'landdergesundheit.de', 'lapekoranera.it', 'lemediapourtous.fr', 'monafm.fr', 'neonrevolt.com', 'positanonotizie.it', 'ronpaullibertyreport.com', 'saobserver.com', 'thetentacle.com', 'transhumanism.news', 'treason.news', 'worlddoctorsalliance.com', 'wwiii.news', 'skeptiko.com', 'globalskywatch.com', 'savethemales.ca', 'trueviralnews.com', 'americanpeopledaily.com', 'teapartypatriots.org', 'act.tv', 'dailydiscord.com', 'theexpose.uk', 'burrardstreetjournal.com', 'thelivingspirits.net', 'jimstone.is', 'davejanda.com', 'theinformedamerican.net', 'conservative-headlines.org', 'americanjournaldaily.com', 'anonhq.com', 'bckonline.com', 'br.sputniknews.com', 'consciousreminder.com', 'corvelva.it', 'cosmetics.news', 'creativehealthyfamily.com', 'detoxed.info', 'discoveries.news', 'dunning-kruger-times.com', 'freedomfictions.com', 'insanity.news', 'lakeshoreadvance.com', 'lancastercourier.com', 'longevity.news', 'newshouse.org', 'nostraightnews.com', 'opioids.news', 'pandoratv.it', 'peachtreetimes.com', 'powergrid.news', 'preparedness.news', 'presstv.com', 'revolt.news', 'rfangle.com', 'rigged.news', 'ru.sputnik.kg', 'theamericanmirror.com', 'thedailyreporter.com', 'tj.sputniknews.ru', 'tyranny.news', 'usdawatch.com', 'onepoliticalplaza.com', 'pdmj.org', 'collective-evolution.com', 'escapeallthesethings.com', 'illuminatiwatcher.com', 'newsview.gr', 'christianaction.org', 'leftaction.com', '12minutos.com', 'en.metapedia.org', 'en.newsner.com', 'knightstemplarorder.com', 'thefrisky.com', 'guarda.one', 'counterpunch.com', 'atlantabusinessjournal.com', 'brigittegabriel.com', 'channel22news.com', 'chocolate.news', 'cz.sputniknews.com', 'dallascitywire.com', 'deepsouthvoice.com', 'deneenborelli.com', 'dmlnewsapp.com', 'eyehealth.news', 'glitch.news', 'healthy-holistic-living.com', 'ilmfeed.com', 'johnplatinumgoss.com', 'londontimes.live', 'lunatics.news', 'oltre.tv', 'poison.news', 'prank.link', 'psycho.news', 'qagg.news', 'sante-nutrition.org', 'ufos.news', 'upliftingtoday.com', 'uz.sputniknews.ru', 'vdmeta.com', 'virtualreality.news', 'voiceofeurope.com', 'americanprinciplesproject.org', 'corona-transition.org', 'oathkeepers.org', 'cloverchronicle.com', 'americandigitalnews.com', 'theproudliberal.org', 'willcountygazette.com', 'nocensura.com', '24aktuelles.com', 'eticamente.net', 'concisepolitics.com', 'darkmoon.me', 'therooster.com', 'archaeology-world.com', 'ahtribune.com', 'altermedzentrum.com', 'conservativepost.com', 'corona-solution.com', 'familiesonline.co.uk', 'fr.news-front.info', 'globebanner.com', 'gunviolence.news', 'healingoracle.ch', 'healthyandnaturalworld.com', 'jimbakkershow.com', 'louisianarecord.com', 'lpheralddispatch.com', 'markzuckerberg.news', 'phillyleader.com', 'pl.sputniknews.com', 'r2020.it', 'rational.news', 'sputniknews.gr', 'timeseportal.com', 'unsere-natur.net', 'uprising.news', 'wholefoods.news', 'gulagbound.com', 'usbacklash.org', 'personalliberty.com', 'americansfortruth.com', 'ihatethemedia.com', 'knowledgeoftoday.org', 'altrighttv.com', 'consciouslifenews.com', 'debka.com', 'familysurvivalheadlines.com', 'lakecountygazette.com', 'commdiginews.com', 'ilmessaggio.it', 'twitter.com/cbugliano', 'madhousemagazine.com', 'ironictimes.com', 'bbspot.com', 'actualite.co', 'adobochronicles.com', '4threvolutionarywar.wordpress.com', 'freedomoutpost.com', 'washingtonsblog.com', 'alexanderhiggins.com', 'financialsurvivalnetwork.com', 'fakingnews.com', 'jewwatch.com', 'skepticink.com', 'facebook.com/womensrightsnews', 'higherperspectives.com', 'redpillpodcasts.com', '209times.com', 'asianexpress.co.uk', 'c19quercetin.com', 'channel45news.com', 'deception.news', 'disinfo.news', 'ecology.news', 'educateinspirechange.org', 'en.news-front.info', 'fastfood.news', 'fbicorruption.news', 'foodsupply.news', 'greatlakeswire.com', 'jedanews.com', 'joebiden.info', 'katehon.com', 'madworldnews.com', 'militarytech.news', 'newsomwatch.com', 'obey.news', 'orientalreview.org', 'oye.news', 'phxreporter.com', 'politicaljournalreview.com', 'qalerts.pub', 'randpaul.news', 'regenere.org', 'remedies.news', 'sheeple.news', 'tg24-ore.com', 'thefullertoninformer.com', 'thehealthyamerican.org', 'wikileaks.com', 'notallowedto.com', 'prisonplanet.com', 'darkjournalist.com', 'climatism.blog', 'fellowshipoftheminds.com', 'cosmicintelligenceagency.com', 'mixi.media', 'other98.com', 'americantruthtoday.com', 'americasfrontlinedoctors.com', 'altright.com', 'cernovich.com', 'filmingcops.com', 'secureamericanow.org', 'communemag.com', 'mchenrytimes.com', 'westcooknews.com', 'breaknotizie.com', 'informare.over-blog.it', 'imao.us', 'landoverbaptist.org', 'disquisendo.wordpress.com', 'notizie-dal-web.it', 'aattp.org', 'dcleaks.com', 'aluminum.news', 'antichrist.news', 'artifacts.news', 'bb4sp.com', 'blacknewschannel.com', 'bugout.news', 'c19melatonin.com', 'c19mp.com', 'c19pvpi.com', 'caffeinamagazine.it', 'channel24news.com', 'civilwar.news', 'coed.com', 'collapsifornia.com', 'corrieredirieti.corr.it', 'cyberwar.news', 'cyborg.news', 'davidharrisjr.com', 'deepstate.news', 'defendevropa.com', 'democide.news', 'depopulation.news', 'derwaechter.net', 'dronewatchnews.com', 'einreich.de', 'emf.news', 'endgame.news', 'energytherapy.biz', 'extra-times.com', 'freepress.news', 'futuresciencenews.com', 'geneticlunacy.com', 'geoengineering.news', 'gmo.news', 'governorpalin.org', 'grandcanyontimes.com', 'groupthink.news', 'healing.news', 'herbs.news', 'indoctrination.news', 'invasionusa.news', 'jameslyonsweiler.com', 'jimhumble.co', 'mindcontrol.news', 'mintpressnews.cn', 'monsanto.news', 'naturalhealth365.com', 'naturalsolutionsmag.com', 'naturecoasttimes.com', 'nigerianwatch.com', 'now8news.com', 'obamawatcher.com', 'ore12.net', 'pesticides.news', 'plague.info', 'portalextras.com', 'potatriotsunite.com', 'puppetstringnews.com', 'radiation.news', 'readingjunkie.com', 'redstatemail.com', 'rofl.news', 'selfdefense.news', 'sidhanepalipost.com', 'southashevillenews.com', 'stopcensura.org', 'thefederalistpapers.org', 'theredelephants.com', 'thinkinghumanity.com', 'thoughtpolice.news', 'vaccinedeaths.com', 'wicatholictribune.com', 'womenshealth.news', 'yetiblog.org', 'yikestoday.com', 'dailybuzzlive.com', 'stormfront.org', 'iceagenow.info', 'infiniteunknown.net', 'usahitman.com', 'ilovemyfreedom.org', 'newnation.org', '10news.one', 'alphanewsmn.com', 'libertyhangout.org', 'staresattheworld.com', 'viralcocaine.com', 'illuminati-news.com', 'rinf.com', '369news.net', 'globalhealingcenter.com', 'climatescienceinternational.org', 'responsibletechnology.org', 'mediaroots.org', 'newsbud.com', 'bluedotdaily.com', 'antifascistnews.net', 'copblock.org', 'gravelinstitute.org', 'thedonald.win', 'msmlies.com', 'conservativepapers.com', 'tplnews.com', 'whatstheharm.net', 'judicialnetwork.com', 'secretnews.fr', 'vnews24.it', 'facebook.com/noncielodicono', 'paperera.com', 'theunrealtimes.com', 'breakingburgh.com', 'southendnewsnetwork.net', 'stiltonsplace.blogspot.com', 'empirenews.net', 'newsexaminer.net', 'actualites.co', 'friendsofsyria.wordpress.com', 'newsjustforyou1.blogspot.com', 'nymeta.co', 'thebreakingnews.co', 'neovitruvian.com', 'aprilamente.info', 'scienzaeconoscenza.it', 'chesuccede.it', 'citymilanonews.com', 'oltrelebarricate.wordpress.com', 'voxnews.online', 'eurosalus.com', 'nuovamedicinagermanica.it', 'attiviamoci.org', 'complottisti.info', 'eyeopening.info', 'infostormer.com', 'nationalufocenter.com', 'thetruthwins.com', 'pamelageller.com', 'politicalblindspot.com', 'aurorasito.wordpress.com', 'ununiverso.altervista.org', 'gotnews.com', 'newobserveronline.com', 'asiannews.it', 'addictinginfo.org', 'ideaspots.com', 'loanpride.com', 'cnoticias.net', 'conservativefighters.com', 'msnbc.co', 'americanconservativemovement.com', 'en.farsnews.com', 'en.radiovaticana.va', 'dailystormer.com', 'vaccinepapers.org', 'beachbroadcastnews.com', 'vaccinesafety.info', '100fedup.com', '100percentfedup.net', '100percentfedup.us', '100percentfedupnews.us', '2020electioncenter.com', '2020maga.pro', '4thgenerationpolitics.com', '8satire.com', '8shit.net', 'abc14news.com', 'abcnews-us.com', 'actionnews3.com', 'activistmommy.com', 'activistpost.org', 'actualidad-rt.com', 'adamscountytimes.com', 'adareporter.com', 'adelaidenationalreview.com', 'aijazeera.com', 'aikentimes.com', 'ajuanews.com', 'akalaat4day.com', 'akronreporter.com', 'albanystandard.com', 'albertanationals.com', 'albusinessdaily.com', 'allaboutusa.pro', 'allenwestrepublic.com', 'alohastatenews.com', 'alterinfo.net', 'alternativemedicine.news', 'alternativenews.com', 'altleft.news', 'altnewsonline.com', 'amarillogazette.com', 'ambientebio.it', 'americafirstprojects.com', 'americanjournalreview.com', 'americannews.pro', 'americanpharmacynews.com', 'americasfrontlinedoctorsummit.com', 'americaslastlineofdefense.com', 'americasnewssource.com', 'americatalks.com', 'amestoday.com', 'andersonreporter.com', 'angolabusinessdaily.com', 'antelopevalleytoday.com', 'anticancer.news', 'antifawatch.news', 'antioxidants.news', 'apopkatimes.com', 'app.thebl.com', 'apparentpolitics.com', 'arborpost.com', 'arizonanewspress24.site', 'arnewsroom.com', 'aroostooknews.com', 'ashevillereporter.com', 'aspartame.news', 'astaxanthin.news', 'athensreporter.com', 'atlstandard.com', 'atrazine.news', 'auburntimes.com', 'austindaily.pro', 'australianmorningherald.com', 'autismovaccini.org', 'autismtruthnews.com', 'awesomeideas.me', 'ayoyeglobal.com', 'aznewsroundup.com', 'baldwintimes.com', 'bannedinformation.com', 'battlecreektimes.com', 'bbc-edition.com', 'bbc-news.us', 'beehivestatenews.com', 'beingtexan.com', 'bentontimes.com', 'berksconews.com', 'biggovernment.news', 'bighealthreport.com', 'bigislandtimes.com', 'bigskytimes.com', 'billnye.news', 'biodefense.com', 'bioterrorism.news', 'bitcoincrash.news', 'bitraped.com', 'blackamericannews.com', 'blendreport.com', 'bloomingtonleader.com', 'blueberries.news', 'bluegrasstimes.com', 'bluestonenews.com', 'blurtdaily.com', 'bnc.tv', 'body2diefor.com', 'boisecitywire.com', 'bordersecurity.news', 'bowlinggreentoday.com', 'bradholiday.com', 'breakingnews247.net', 'breakingnews365.net', 'breakingnewshouse.com', 'brevardsun.com', 'bridgeporttimes.com', 'btgnetworks.com', 'buffalochronicle.online', 'burlingtonreporter.com', 'burlingtonstandard.com', 'businessglitz.com', 'butam.net', 'c-vine.com', 'c19bromhexine.com', 'c19budesonide.com', 'c19colchicine.com', 'c19curcumin.com', 'c19ic.com', 'c19ly.com', 'c19sv.com', 'califodaily.club', 'californiacollapse.news', 'californianewspress24.site', 'caliuspost.site', 'campusinsanity.com', 'canadiantimes.live', 'cancercriminals.com', 'cancermyths.com', 'cancerscams.com', 'cancersolutions.news', 'cannabiscures.news', 'capecodledger.com', 'capemonews.com', 'carbondioxide.news', 'cbds.news', 'cbinfo24.com', 'cbs-news.us', 'cbsnews10.com', 'cbsnews15.com', 'cbtvn.com', 'cdareporter.com', 'cedarrapidstoday.com', 'celbmag.com', 'celebrityreputation.com', 'censoredscience.com', 'centralbrowardnews.com', 'centralbuckstoday.com', 'centralchestertoday.com', 'centralcoloradonews.com', 'centralgeorgianews.com', 'centraliowatimes.com', 'centrallouisiananews.com', 'centralmontanatimes.com', 'centralndnews.com', 'centraloctimes.com', 'centralpanews.com', 'centralshenandoahnews.com', 'centraltxnews.com', 'centralutahnews.com', 'centralwisconsinnews.com', 'centralwynews.com', 'centroplexnews.com', 'channel23news.com', 'channel28news.com', 'channel34news.com', 'channel40news.com', 'chaospost.com', 'chapelhillreview.com', 'charlestonreporter.com', 'charlottedaily.pro', 'chemo.news', 'chemotherapy.news', 'chicagodaily.pro', 'chicotimes.com', 'chlorella.news', 'chlorellafactor.com', 'chryslernews.com', 'cisiamo.info', 'cistranfinance.com', 'citizensjournal.substack.com', 'clarksvilletimes.com', 'claycotimes.com', 'clayreporter.com', 'cleanwater.news', 'clearnewswire.com', 'clermonttimes.com', 'clevelandreporter.com', 'clpolitico.xyz', 'cnbsnews.live', 'coachellatoday.com', 'coastalganews.com', 'cobbreporter.com', 'cobusinessdaily.com', 'coconinonews.com', 'coconutoil.com', 'coinglobalist.com', 'cole24info.com', 'collegeparktoday.com', 'coloradopress91.site', 'columbiastandard.com', 'columbusdaily.site', 'columbusnews21.site', 'columbusnews22.site', 'columbusnews23.site', 'columbusnews32.site', 'columbusnews35.site', 'columbuspress91.site', 'columbusstandard.com', 'columbususapost.site', 'columbususpost.site', 'compilenews.com', 'conservative101.com', 'conservativematrix.com', 'conservativept.com', 'conservativetears.com', 'conservativewiredaily.com', 'constitutionstatenews.com', 'coreybooker.news', 'coronavirusoutbreakmap.com', 'counterthink.com', 'covid19up.org', 'covidglobalnews.live', 'crawfordtimes.com', 'crazyifonews.com', 'crybullies.news', 'ctbusinessdaily.com', 'cubicalnews.com', 'cukam.com', 'cukaminfo.com', 'cukammedia.com', 'cvikas.com', 'cvikasdrv.com', 'cvrcinfo.com', 'd20news.com', 'daily-inquirer.com', 'dailyncb.com', 'dailypoliticalnews.info', 'dailystarpost.com', 'dailyworldupdate.us', 'dakotanews.pw', 'dan-news.info', 'dangerousmedicine.com', 'davidgorski.news', 'davistimes.com', 'dayreports.xyz', 'daytonreporter.com', 'dcdailywire.com', 'dctribune.org', 'de.news-front.info', 'decaturtimes.com', 'deepstatejournal.com', 'dekalbganews.com', 'delcoreview.com', 'democracyawakensinaction.com', 'democraticmoms.com', 'denvercitywire.com', 'denverguardian.com', 'desmoinessun.com', 'detox.news', 'detroitdaily.xyz', 'deutschland-kurier.org', 'dhmakanews.xyz', 'diabetescure.news', 'diabetessciencenews.com', 'diaryfast.com', 'dicamba.news', 'digynews.com', 'disrn.com', 'dissept.com', 'distributednews.com', 'district2news.com', 'dmaga2020.xyz', 'doctormurray.com', 'doctorphillipstoday.com', 'domesticreport.com', 'domesticterrorism.com', 'dorchestertoday.com', 'dotinfo24.com', 'downtrend.com', 'drivenpost.com', 'drsergegregoire.com', 'drugcartels.news', 'dsmsportswire.com', 'dtldrr.pro', 'dubuquetimes.com', 'durhamreporter.com', 'duvaltimes.com', 'eaglerising.com', 'earlyview.net', 'eastalamedanews.com', 'eastarapahoenews.com', 'eastarizonanews.com', 'eastcontracostanews.com', 'easterislandstatues2019.com', 'easternwaynetoday.com', 'easthillsboroughnews.com', 'easthoustonnews.com', 'eastidahotimes.com', 'eastindynews.com', 'eastkentuckytimes.com', 'eastlittlerocktimes.com', 'eastlouisvillenews.com', 'eastpanhandlenews.com', 'eastpanhandletimes.com', 'eastsandiegonews.com', 'eastsbvtimes.com', 'eastsfvtoday.com', 'eastsierranews.com', 'eastslcnews.com', 'eaststarknews.com', 'easttwincities.com', 'eastventuranews.com', 'eastvolusianews.com', 'eastwaketimes.com', 'ecalabamanews.com', 'ecgeorgianews.com', 'ecindiananews.com', 'eciowanews.com', 'eckansasnews.com', 'ecmissnews.com', 'ecnebraskanews.com', 'ecohionews.com', 'ecwisconsinnews.com', 'educationsystem.news', 'ef-magazin.de', 'elkharttimes.com', 'elonmuskwatch.com', 'emeraldcoasttimes.com', 'emergencymedicine.news', 'eminetra.co.nz', 'eminetra.co.uk', 'eminetra.com', 'en-volve.com', 'entertainme.whatfinger.com', 'epnewswire.com', 'epochtimes.ru', 'ericthompsonshow.com', 'esports.yahoo.com', 'esrt.tech', 'eugenics.news', 'euthanasia.news', 'everylegalvote.com', 'evilgoogle.news', 'explainlife.com', 'expressnews.pw', 'fabiosa.com', 'fabiosa.guru', 'fabiosa.me', 'fabiosanews.com', 'facebookcollapse.com', 'factswt.com', 'fakepandemic.com', 'fakescience.news', 'fakeshare.com', 'falseflag.news', 'fanmagazine.it', 'fargostandard.com', 'fayettevillestandard.com', 'fbnewscycle.com', 'fdareporter.com', 'feednets.com', 'feednewz.com', 'femalista.com', 'findlaytimes.com', 'fingerlakestoday.com', 'firststatetimes.com', 'firstwalkup.net', 'fishoils.news', 'flinkfeed.com', 'floridagazette.pro', 'floridanewspress24.site', 'floridaparishnews.com', 'flushot.news', 'flusurvival.com', 'flyingcars.news', 'follownews.pw', 'fondulacnews.com', 'foodcures.news', 'foodfreedom.news', 'foodismedicine.com', 'foodismedicine.news', 'foodrising.org', 'foodscience.news', 'forfreedomworld.com', 'fortsmithtimes.com', 'fox-32chicago.com', 'foxcitiesnews.com', 'foxhole.news', 'foxnews-us.com', 'foxnews15.com', 'foxnewsinsider.com', 'fr.theepochtimes.com', 'francenewslive.com', 'free-speechfront.info', 'freepressfront.com', 'freespeech-time.com', 'freespeechfront.net', 'freie-medien.tv', 'french.presstv.com', 'fresnoleader.com', 'ftbagency.com', 'ftwaynetimes.com', 'futuretech.news', 'gadsdentoday.com', 'gatewayreporter.com', 'gemstatewire.com', 'genderconfused.com', 'geneticlunancy.com', 'geniusget.com', 'georgiadailypost.site', 'georgiamountainnews.com', 'georgianewspress36.site', 'getnewstuff.com', 'glaciercountrynews.com', 'globalnews4u.xyz', 'globalnewspros.xyz', 'globalremedyhouse.com', 'glyphosate.news', 'gnfnews.com', 'godfatherpolitics.com', 'goeportal.com', 'goldcoastreview.com', 'goldcountrytoday.com', 'goldengatesports.com', 'goldenstatetoday.com', 'govtslaves.com', 'grandjunctiontimes.com', 'grandrapidsreporter.com', 'greatamericanrepublic.com', 'greatertexan.com', 'greenbayreporter.com', 'greenlivingnews.com', 'greenmedinfo.health', 'gretawire.foxnewsinsider.com', 'grocerycures.com', 'growyourmedicine.com', 'guchills.com', 'gucmakale.com', 'hancevillenews.com', 'happymemorialday2019.com', 'harfordnews.com', 'hartfordreporter.com', 'hawkeyereporter.com', 'hcqrct.com', 'hcworldnews.com', 'healingarts.news', 'healthandmoneynews.wordpress.com', 'healthcoverage.news', 'healthfreedom.news', 'healthrangerscience.com', 'healthylifeboxx.com', 'heavymetals.news', 'hempscience.news', 'henricotimes.com', 'herbs-info.com', 'hernandoreporter.com', 'hibusinessdaily.com', 'hickorysun.com', 'highcountrytimes.com', 'hillcountrychronicle.com', 'hiltonheadreporter.com', 'hindstoday.com', 'holisticdentistry.news', 'hollandreporter.com', 'hollywoodhater.com', 'homebuilders.space', 'homegardeningnews.com', 'homesteading.news', 'honolulureporter.com', 'hopkinsvilletimes.com', 'hotgazette.com', 'hotspringstimes.com', 'houmathibodauxnews.com', 'housatonicvalleynews.com', 'howardconews.com', 'howtodofor.com', 'hudsontoday.com', 'hundredfedup.staging.wpengine.com', 'hunterscreeknews.com', 'huntsvilleleader.com', 'huzlers.com', 'huzlersmedia.com', 'hzeppfeed.com', 'illicitinfo.com', 'ilovemyamerica.net', 'immunization.news', 'imperialcanews.com', 'indexexpurgatorius.wordpress.com', 'informarexresistere.fr', 'informasalus.it', 'infosco.org', 'innerstrength.zone', 'interioralaskanews.com', 'intolerance.news', 'introtimes.com', 'inventions.news', 'iodine.news', 'iolgue.com', 'iowacitytoday.com', 'ir.sputniknews.com', 'jacksonpurchasenews.com', 'jamescomey.news', 'jedanews.it', 'jeffbezoswatch.com', 'jeffcotimes.com', 'jeffersonreporter.com', 'jeffrainforth.com', 'jessaminejournal.com', 'jesuisfrancais.net', 'jesusdaily.com', 'jocotoday.com', 'johnscreekpost.com', 'johnstonreporter.com', 'jonesborotimes.com', 'justicedemocrats.news', 'kalamazootimes.com', 'kauaisun.com', 'kckingdom.com', 'kenoshareporter.com', 'kentcountytoday.com', 'kentuckianatimes.com', 'kentuckypress31.site', 'kerncountytimes.com', 'kevkainfo.com', 'keywestreporter.com', 'khabar660media.com', 'kingscountytimes.com', 'kingsportreporter.com', 'knoxtimes.com', 'koinesune.com', 'kokomostandard.com', 'labusinessdaily.com', 'lafayettereporter.com', 'lafayettetimes.com', 'lagauchematuer.fr', 'laharbornews.com', 'lakebutlernews.com', 'lakecountytimes.com', 'laketahoesun.com', 'laminutebienetre.net', 'lansingreporter.com', 'lansingsun.com', 'lapress.pro', 'larimernews.com', 'lastfrontiernews.com', 'lastresistance.com', 'lasvegasrecord.com', 'laurelhighlandstoday.com', 'lawrencetimes.com', 'laxleader.com', 'ledinews.com', 'leetoday.com', 'leftcult.com', 'legitim.ch', 'lewistontimes.com', 'libertyfrontpress.com', 'libertyinfonews.com', 'liberyinfo24.com', 'libtards.news', 'liebeisstleben.net', 'linfolibre.com', 'litchfieldhillstoday.com', 'liverdamage.news', 'livingfree.news', 'lizwarren.news', 'lostmisfits.com', 'louisvillecitywire.com', 'lowedeltanews.com', 'lowerbuckstoday.com', 'lowerwestscnews.com', 'lt.sputniknews.ru', 'lubbocktimes.com', 'luzernetimes.com', 'lycomingnews.com', 'lynchburgreporter.com', 'macombtoday.com', 'macontimes.com', 'madisonreporter.com', 'maga2020.press', 'maga2020.pro', 'magadailyreport.com', 'magafnb.press', 'magakagnews.com', 'maganews.pro', 'magicvalleytimes.com', 'magnanewspapers.com', 'magnoliastatenews.com', 'magrebnewswire.com', 'mainebusinessdaily.com', 'maineexaminer.com', 'mainehighlandsnews.com', 'manateereview.com', 'manilabusinessdaily.com', 'mansfieldtimes.com', 'marialourdesblog.com', 'marinleader.com', 'marketcrash.news', 'marylandnews9.site', 'matsutimes.com', 'matyinfo.com', 'mauireporter.com', 'maxnews.pro', 'mb.ntd.tv', 'mdbusinessdaily.com', 'mdstatewire.com', 'measlesnews.com', 'mediafactwatch.com', 'medicalextremism.com', 'medicalfacts.it', 'medicalmarijuanaupdate.com', 'medicaltyranny.com', 'medicine-today.net', 'medinatoday.com', 'mednat.news', 'mednat.org', 'megadealernews.com', 'mensfitnessfocus.com', 'menshealth.news', 'mercedtimes.com', 'mercurysciencenews.com', 'mesapress91.site', 'metricmedianews.com', 'metrolexnews.com', 'metrowesttimes.com', 'mexicobusinessdaily.com', 'miamicourant.com', 'mibusinessdaily.com', 'micapitolnews.com', 'michigannews9.site', 'microwavetinfoil.com', 'midcitytimes.com', 'midcoasttoday.com', 'middleamericandemocrat.com', 'midmassnews.com', 'midvalleyreporter.com', 'migrantsonthemove.org', 'mihogarnatura.com', 'mindbodyscience.news', 'mipt.co', 'missouridownload.com', 'mkenorth.com', 'mkesouth.com', 'mobilecourant.com', 'mobusinessdaily.com', 'mohavetoday.com', 'money.whatfinger.com', 'monroeconews.com', 'monroenynews.com', 'monroereview.com', 'monsantomafia.com', 'monse.club', 'montereytimes.com', 'montgomerymdnews.com', 'moon-child.net', 'morrisleader.com', 'mountainstatetimes.com', 'msbusinessdaily.com', 'msgulfnews.com', 'mtbusinessdaily.com', 'munciereporter.com', 'muskegonsun.com', 'mydeeptruth.com', 'myliberty.one', 'mymilitia.com', 'n23.tv', 'nantahalanews.com', 'naplesstandard.com', 'nashvillestandard.com', 'nassaustandard.com', 'nationaldebt.news', 'nationalistreview.net', 'naturalcures.news', 'naturalhealth.news', 'naturalmedicine.news', 'naturalnewsradio.com', 'naturalpedia.com', 'naturalstatenews.com', 'naturopathy.news', 'naturstoff-medizin.de', 'naugatucktimes.com', 'nbc-news.org', 'nbc9news.com', 'nbcnews6.com', 'ncarkansasnews.com', 'ncfloridanews.com', 'ncgeorgianews.com', 'ncindiananews.com', 'nckansasnews.com', 'nckentuckynews.com', 'ncmassnews.com', 'ncmissnews.com', 'ncncnews.com', 'ncpatimes.com', 'ncsctimes.com', 'ncunionnews.com', 'ncwisconsinnews.com', 'ncwvnews.com', 'ndbusinessdaily.com', 'nealabamanews.com', 'neatlantanews.com', 'nebraskabusinessdaily.com', 'necalinews.com', 'necoloradonews.com', 'neconnnews.com', 'nefloridanews.com', 'negronews.fr', 'neiltyson.news', 'neindiananews.com', 'neiowanews.com', 'nekansasnews.com', 'nekentuckynews.com', 'nemissnews.com', 'nemissourinews.com', 'nenashvillenews.com', 'nenebraskanews.com', 'nenewmexiconews.com', 'nenorthdakotanews.com', 'neohiotimes.com', 'nepanhandlenews.com', 'nesacramentonews.com', 'nesdnews.com', 'netarrantnews.com', 'nevadanews9.site', 'nevadapost365.site', 'nevalleytimes.com', 'newashingtonnews.com', 'newenergyreport.com', 'newisconsinnews.com', 'newnews40.pro', 'news.konbini.com', 'news3insider.com', 'news4ktla.com', 'news91.me', 'newscartels.com', 'newschant.com', 'newsclub.pro', 'newsfakes.com', 'newsfeedin.com', 'newsinsider-us.com', 'newsmag.pro', 'newsmedia2020.pro', 'newsrapter.com', 'newsroom24.co.uk', 'newsservis.com', 'newstab.us', 'newstv3.com', 'newyorkpost.com', 'newzealandtimes.live', 'nhbusinessdaily.com', 'niagaraleader.com', 'nndnews.pro', 'nnettle.com', 'nortextimes.com', 'northacadiananews.com', 'northalaskanews.com', 'northbaltimorejournal.com', 'northbirminghamtimes.com', 'northbluegrassnews.com', 'northbrowardnews.com', 'northchiltonadvertiser.com', 'northcoastalnews.com', 'northcoastcanews.com', 'northcountryleader.com', 'northcountryreporter.com', 'northdelconews.com', 'northdsmnews.com', 'northerntiernews.com', 'northessexnews.com', 'northfultontoday.com', 'northguilfordnews.com', 'northhennepinnews.com', 'northidahotimes.com', 'northindynews.com', 'northinlandnews.com', 'northjeffconews.com', 'northkentuckynews.com', 'northlaketimes.com', 'northlancasternews.com', 'northlittlerocktimes.com', 'northmianews.com', 'northmichigannews.com', 'northnewcastlenews.com', 'northoctimes.com', 'northokcnews.com', 'northorlandonews.com', 'northpalmbeachtoday.com', 'northpanhandlenews.com', 'northpimanews.com', 'northpinellasnews.com', 'northpocononews.com', 'northraleightoday.com', 'northramseynews.com', 'northsactoday.com', 'northsanantonionews.com', 'northsfvtoday.com', 'northsgvnews.com', 'northsnohomishnews.com', 'northsummitnews.com', 'northtrianglenews.com', 'northtxnews.com', 'northutahnews.com', 'northwichitanews.com', 'northwoodsreporter.com', 'nosignalfound.fr', 'nosignalfound894736876.wordpress.com', 'novitimes.com', 'nrcnews.pro', 'nsfnews.com', 'ntdr34.pro', 'nthqldnationalreview.com', 'ntlddrr.com', 'ntldrr.pro', 'nutraceuticals.news', 'nwalabamanews.com', 'nwarkansasnews.com', 'nwbergennews.com', 'nwclarknews.com', 'nwconnnews.com', 'nwfranklinnews.com', 'nwgeorgianews.com', 'nwhoustonnews.com', 'nwiowanews.com', 'nwkansasnews.com', 'nwlatimes.com', 'nwminnesotanews.com', 'nwmissnews.com', 'nwmontananews.com', 'nwohiotimes.com', 'nwpanews.com', 'nwpedia.club', 'nwriversidenews.com', 'nwvalleytimes.com', 'nwwashingtonnews.com', 'nwwaynenews.com', 'nwwyomingnews.com', 'nybusinessdaily.com', 'nycgazette.com', 'nylive.me', 'nynews.pro', 'nytwatch.com', 'oaklandrecord.com', 'objectivist.co', 'ocalastandard.com', 'oceancountyleader.com', 'oceanhealthnews.com', 'oceanstatetoday.com', 'off-guardian.org', 'offgrid.news', 'ohiopress.pro', 'okeechobeetimes.com', 'olympictimes.com', 'omega3.news', 'one-voice.fr', 'oneworld.press', 'onslownews.com', 'operationq.pub', 'opreminfo.com', 'opremmedia.com', 'oregonnewspress.com', 'oregonnewspress36.site', 'oregonnewspress53.site', 'oregonnewspress56.site', 'oregonnewspress58.site', 'oregonnewspress59.site', 'oregonnewspress61.site', 'oregonnewspress63.site', 'oregonnewspress99.site', 'organharvesting.news', 'organicfarming.news', 'organics.news', 'orlandostandard.com', 'ourdavie.com', 'outerbankstimes.com', 'outragedepot.com', 'oz.sputniknews-uz.com', 'palmcoasttimes.com', 'palmettobusinessdaily.com', 'palmettostatenews.com', 'panamacityreporter.com', 'part2-njfm.crd.co', 'part3-njfm.crd.co', 'pascoreporter.com', 'patrioticexpress.com', 'patriotnetwork.one', 'pcsolottoresult4d.com', 'peacedata.net', 'peacegardennews.com', 'pegadinha.link', 'pensacolatimes.com', 'peternews.xyz', 'petfoodscience.com', 'petfoodwarning.com', 'pethealth.news', 'pharmaceuticalfraud.com', 'phoenixdaily.club', 'phoenixnews21.site', 'phoenixnews25.site', 'phoenixnews33.site', 'phoenixnews5.site', 'phoenixnews9.site', 'phoenixusapost.site', 'phoenixusnews.site', 'phoenixuspost.site', 'phytonutrients.news', 'pi-news.net', 'pinaltoday.com', 'pinebelttimes.com', 'pinellastimes.com', 'pinestatenews.com', 'pittreview.com', 'plandemicmovie.com', 'policestate.news', 'political-discussion.com', 'politicalcorrectness.news', 'politicalpalace.com', 'politicsmagic.com', 'politicsonline.net', 'politicsspot.com', 'politicsupdates.pro', 'politifeed.net', 'polktimes.com', 'pomonavalleynews.com', 'populationcollapse.com', 'populationcontrol.news', 'portagetimes.com', 'portlandcourant.com', 'poseidonnews.com', 'postnewsera.com', 'potatriotpost.us', 'powderedwigsociety.com', 'powerfulreport.com', 'prankyourfriends.com', 'pravdareport.com', 'prcmediainfo.com', 'prentissheadlight.com', 'presspoint24.com', 'pridemagazine.com', 'privacywatch.news', 'prntly.com', 'propertyinsurancewire.com', 'protegetasante.net', 'psychdrugs.news', 'psychdrugwatch.com', 'pulaskitimes.com', 'qactusinformateur.wordpress.com', 'qalerts.net', 'qanon.app', 'qdrop.pub', 'qnation.us', 'qposts.online', 'quackery.news', 'r2020.info', 'racewar.news', 'racinesun.com', 'radiationscience.com', 'randolphcountyheraldtribune.com', 'rawfood.news', 'rcidnews.pro', 'realdaily.pro', 'realinvestigations.news', 'realsciencenews.com', 'record-bee.com', 'redbluedivide.com', 'reddingtoday.com', 'redefiningsocial.com', 'redwoodempirenews.com', 'regardecettevideo.fr', 'renoreporter.com', 'republicandaily.net', 'resistancerepublicaine.eu', 'reverbpress.site', 'reverseheartdisease.news', 'richardpan.news', 'richmondleader.com', 'rightwingpost.com', 'rightwingtribune.com', 'riverbendtimes.com', 'riverparishnews.com', 'riverregiontimes.com', 'rivervalleytoday.com', 'ro.sputnik.md', 'roanokesun.com', 'rocklandreporter.com', 'rs.sputniknews.com', 'ru.sputnik.md', 'rulebysecrecy.com', 'runwonder.com', 'sacorridornews.com', 'sacramentopress91.site', 'sacramentostandard.com', 'sanantoniostandard.com', 'sandhillstoday.com', 'sandiegorecord.com', 'sanfransun.com', 'sanjoaquintimes.com', 'sanjosestandard.com', 'sanmateosun.com', 'santaclaratoday.com', 'santacruzstandard.com', 'sarasotareview.com', 'scalaskanews.com', 'scconnnews.com', 'scienceclowns.com', 'sciencedeception.com', 'sciencefraud.news', 'sciencetyranny.com', 'scitech.whatfinger.com', 'sckansasnews.com', 'scnebraskanews.com', 'scoklahomanews.com', 'scpanews.com', 'scrantonreporter.com', 'sctexasnews.com', 'scvermontnews.com', 'sdbusinessdaily.com', 'sealaskanews.com', 'sealleghenynews.com', 'searizonanews.com', 'seatlantanews.com', 'seattledaily.site', 'secoloradonews.com', 'secondamendment.news', 'secondamendmentdaily.com', 'seconnnews.com', 'sedallasnews.com', 'sedenvernews.com', 'segnidalcielo.it', 'sehoustonnews.com', 'seindiananews.com', 'seiowanews.com', 'sekansascitynews.com', 'selatimes.com', 'semissourinews.com', 'semontananews.com', 'senebraskanews.com', 'seoaklandnews.com', 'seoklahomanews.com', 'setexastimes.com', 'setwincities.com', 'sewyomingnews.com', 'sfvtoday.com', 'sgvstandard.com', 'shenandoahvalleynews.com', 'shenangovalleynews.com', 'shoalstoday.com', 'shortme.us', 'showmestatetimes.com', 'shredmoney.com', 'shtf.news', 'siouxcitytimes.com', 'skeptics.news', 'skytg24news.it', 'slctimes.com', 'slender.news', 'sloreporter.com', 'smeared.news', 'smokymountaintoday.com', 'snopes.news', 'socialjustice.news', 'solanosun.com', 'somersettimes.com', 'southabqnews.com', 'southalabamatimes.com', 'southalamedanews.com', 'southatlantanews.com', 'southbayleader.com', 'southbaysdnews.com', 'southbendtimes.com', 'southbergennews.com', 'southbirminghamtimes.com', 'southbrowardnews.com', 'southchestertoday.com', 'southcotimes.com', 'southcumberlandnews.com', 'southcuyahoganews.com', 'southdfwnews.com', 'southernindianatoday.com', 'southernwvnews.com', 'southfairfaxnews.com', 'southfranklinnews.com', 'southfultontoday.com', 'southhennepinnews.com', 'southindynews.com', 'southiowanews.com', 'southjeffconews.com', 'southkingnews.com', 'southlaketoday.com', 'southlouisiananews.com', 'southmainenews.com', 'southmianews.com', 'southncnews.com', 'southnewcastlenews.com', 'southoctimes.com', 'southomahatimes.com', 'southorlandonews.com', 'southpalmbeachtoday.com', 'southpimanews.com', 'southpinellastimes.com', 'southraleighnews.com', 'southsactoday.com', 'southsanantonionews.com', 'southsfbaynews.com', 'southsfvtoday.com', 'southsgvnews.com', 'southsidevanews.com', 'southstlnews.com', 'southtidewaternews.com', 'southtrianglenews.com', 'southtulsatoday.com', 'southwichitanews.com', 'spacetourism.news', 'speech-point.com', 'speech-point.net', 'spirulina.news', 'spokanecotimes.com', 'spokanestandard.com', 'sports.whatfinger.com', 'springfieldstandard.com', 'springstimes.com', 'sputnik-abkhazia.info', 'sputnik-georgia.com', 'sputnik-tj.com', 'sputnik.md', 'sputniknews-uz.com', 'sputniknews.lt', 'sputniknewslv.com', 'stanislausnews.com', 'statins.news', 'stcharlestimes.com', 'stcloudsun.com', 'stlwestnews.com', 'stopcensura.info', 'stopcovidcold.com', 'stopeatingpoison.com', 'stopmandatoryvaccination.com', 'stopsmoking.news', 'stpetestandard.com', 'sturgiscoldwaternews.com', 'sumtertimes.com', 'sunday-herald.com', 'sunflowerstatenews.com', 'sunshinesentinel.com', 'superbugs.news', 'superfood.news', 'superfoods.news', 'superfoodsnews.com', 'supplementsreport.com', 'sussexreview.com', 'swalaskanews.com', 'swarizonanews.com', 'swarkansastimes.com', 'swbluegrassnews.com', 'swcoloradonews.com', 'swconnnews.com', 'sweeteners.news', 'swgeorgianews.com', 'swhoustonnews.com', 'swiowatimes.com', 'swkansasnews.com', 'swlouisiananews.com', 'swnewhampshirenews.com', 'swriversidenews.com', 'swvalleytimes.com', 'swvegasnews.com', 'swvirginianews.com', 'sydneynewstoday.com', 'syracusesun.com', 'takebackour.world', 'talknetwork.com', 'tallahasseesun.com', 'tamche1966.com', 'tamparepublic.com', 'tapainfo.com', 'tapwater.news', 'taterforceone.com', 'tatersgonnatate.com', 'technocrats.news', 'texasmag.pro', 'texasnews9.site', 'texaspost.pro', 'texaspost365.site', 'texaspress31.site', 'tfppnews.com', 'tftpnews.com', 'the-dailystar.com', 'the-southeast-journal.com', 'theantimedia.com', 'theblackloop.com', 'thecoloradoherald.com', 'theconservativeoutlet.com', 'thefakenewsgenerator.com', 'thefloridapost.com', 'thegoodlordabove.com', 'theheartysoul.com', 'thelibertyeagle.com', 'themagaheroes.com', 'themichiganstar.com', 'thenationalmarijuananews.com', 'thepolitics.online', 'theragingpatriot.com', 'theragingpatriot.net', 'theragingpatriot.org', 'thescienceandspace.com', 'thescinewsreporter.com', 'thescoopradioshow.com', 'thetrumpnewsgazette.com', 'thetruthvoice.net', 'theunitedstatesblues.com', 'thevoteinfo.com', 'thewashingtonpundit.com', 'thewashingtonsentinel.com', 'thewashingtontime.com', 'theweeklyobserver.com', 'thimerosal.news', 'threalnews.net', 'time48h.com', 'tmaga2020.pw', 'todayupdates.pro', 'tonmag.fr', 'top20radio.tv', 'topekasun.com', 'topibuzz.me', 'topsante.net', 'torontobusinessdaily.com', 'touchalpha.com', 'traitors.news', 'treasurecoastsun.com', 'treasurevalleytimes.com', 'trendingstylist.com', 'trendyfeed.net', 'tricitiesreporter.com', 'tricitysun.com', 'trump-feed.com', 'truthclip.com', 'truthfeednews.com', 'truthmed.social', 'truthuncensored.net', 'truthunsealed.com', 'truthwiz.com', 'tucsonstandard.com', 'tularetimes.com', 'tulsastandard.com', 'turmeric.news', 'tuscaloosaleader.com', 'txrealtynews.com', 'uberbrief.com', 'ukhealthexpert.com', 'unclesamsmisguidedchildren.com', 'unexplained.news', 'unioncountyreview.com', 'universaleinfo.com', 'universalemedia.net', 'unzensuriert.de', 'upmnetwork.com', 'upperbuckstoday.com', 'uppercumberlandtimes.com', 'upperdeltanews.com', 'uppereasttx.com', 'upperwestscnews.com', 'upstatescnews.com', 'usa2019news.blogspot.com', 'usareally.com', 'usataters.com', 'usatodaypost.com', 'uschronicle.com', 'uslibertywire.com', 'usrightleft.com', 'ustruthwire.com', 'utahvalleytimes.com', 'vacationlandtimes.com', 'vaccination.co.uk', 'vaccinedamage.news', 'vaccinefromhell.com', 'vaccineholocaust.org', 'vaccinejihad.com', 'vaccineriskawareness.com', 'vaccineshed.com', 'vancouverreporter.com', 'veggie.news', 'verdugosnews.com', 'ververeport.com', 'veteranstoday.news', 'victorvalleytimes.com', 'video.lasentinella.gelocal.it', 'video.latino.foxnews.com', 'vietonique.com', 'viralhatch.com', 'viralmagazine.it', 'viralvirus.de', 'virginiapress31.site', 'vitamind.news', 'vn.sputniknews.com', 'vonjour.fr', 'votedemocrat.news', 'votefraud.news', 'voterepublican.news', 'vtbusinessdaily.com', 'vulcanadvocate.com', 'w24n.com', 'wabusinessdaily.com', 'waislenews.com', 'wapoop.news', 'warnerrobinstoday.com', 'warrenclintonnews.com', 'washingtondailywire.com', 'washingtonnews9.site', 'washingtonposted.news', 'washingtonpress.pro', 'washingtontimes.pro', 'watchesmam.com', 'watchuncensored.com', 'waterfilters.news', 'waterfordtoday.com', 'waterlootimes.com', 'waterpurifiers.news', 'waukeshatimes.com', 'waynecountytoday.com', 'wcalabamanews.com', 'wciowanews.com', 'wcmichigannews.com', 'wcmissourinews.com', 'wcnewmexiconews.com', 'wcpanews.com', 'weagreeinfo.com', 'wearethellod.com', 'webdaily.com', 'webertimes.com', 'weeklyinquirer.com', 'weirdsciencenews.com', 'weirdworldinfo.com', 'wellbeingjournal.com', 'westbanklanews.com', 'westchesterreporter.com', 'westclevelandnews.com', 'westcontracostanews.com', 'westdfwnews.com', 'westdsmnews.com', 'westeldoradonews.com', 'westernndnews.com', 'westernsdnews.com', 'westessexnews.com', 'westflnews.com', 'westhamiltonnews.com', 'westhillsboroughnews.com', 'westindynews.com', 'westlatimes.com', 'westnovanews.com', 'westoctimes.com', 'westpennyroyalnews.com', 'westsbvtimes.com', 'westsfvtoday.com', 'westsgvnews.com', 'westslcnews.com', 'weststlnews.com', 'westventuranews.com', 'westvolusianews.com', 'westwakenews.com', 'whitenews.pro', 'whitewatertimes.com', 'wichitastandard.com', 'wikr.com', 'williamsburgsun.com', 'winecountrytimes.com', 'wiregrasstimes.com', 'wlinks.co', 'wokesloth.com', 'womensfitnessfocus.com', 'wonderneed.com', 'world5news.com', 'wvheartlandnews.com', 'wyandottetimes.com', 'wybusinessdaily.com', 'xmaga2020.site', 'yadkinvalleynews.com', 'yavapainews.com', 'yellowhammertimes.com', 'yestardaygone.online', 'yllnews.pw', 'yogurt.news', 'yorkdaily.site', 'youngstowntimes.com', 'yourdailypulse.com', 'youtubecensorship.com', 'yubasuttertimes.com', 'zeppfeed.com', 'angrypatriotmovement.com', 'activepost.com', 'newsbreakshere.com', 'americannews.com', 'thenewyorkevening.com', 'socialeverythings.com', 'react365.com', 'freedomdaily.com', 'thedailysheeple.com', 'revolutionradio.org', 'sheepkillers.com', 'thoughtcrimeradio.net', 'worldaffairsbrief.com', 'assassinationscience.com', 'consumerwellness.org', 'earthpulse.com', 'amtvmedia.com', 'dailyoccupation.com', 'freedomadvocates.org', 'newamericannews.com', 'rapturenewsnetwork.com', 'realnews24.com', 'godandscience.org', 'getholistichealth.com', 'hsionline.com', 'libertyvideos.org', 'goodgopher.com', 'imowired.com', 'lindelltv.com', 'natall.com', 'blingnews.com', 'researchantisemitism.ca', 'dailyheadlines.net', 'dcgazette.com', 'hangthebankers.com', 'hoggwatch.com', 'profam.org', 'myrightamerica.com', 'geopol.institute', 'newsheist.com', 'politifact.news', 'steadfastandloyal.com', 'stonecoldtruth.com', 'triggerreset.net', 'truthinmedia.com', 'yesimright.com', 'healingfoodreference.com', 'herbreference.com', 'nutrientreference.com', 'supplementreference.com', 'healthranger.com', 'conservativebeaver.com', 'grainoftruth.ca', 'intellihub.com', 'thespectator.info', 'usaisonline.com', 'conservativeammo.com', 'fortherightnews.com', 'inforos.ru', 'leftexposed.org', 'libertybell.com', 'navarroreport.com', 'newspushed.com', 'pcmustdie.com', 'righterway.com', 'swampdrain.com', 'theconservativecentral.com', 'thelaughclub.net', 'torontotoday.net', 'newsfeedhunter.com', 'empireherald.com', 'dataasylum.com', 'viralnewsnetwork.net', 'vaccinesrevealed.com', 'cityworldnews.com', 'cap-news.com', 'thinkingmomsrevolution.com', 'pakalertpress.com', 'stormcloudsgathering.com', 'whydontyoutrythis.com', 'ifyouonlynews.com', 'theforbiddenknowledge.com', 'londonwebnews.com', 'loonwatch.com', 'greenvillegazette.com', 'thedailyconspiracy.com', 'squawker.org', 'patriotfires.com', 'cnn-globalnews.com', 'nationaleconomicseditorial.com', 'globemagazine.com', 'haarp.net', 'autisminvestigated.com', 'ewao.com', 'acallforanuprising.com', 'allianceforadvancedhealth.com', 'freedomainradio.com', 'ourhealthguides.com', 'anh-usa.org', 'naturalcures.com', 'livingwhole.org', 'modernalternativemama.com', 'naturalawakeningsmag.com', 'occupyyourself.org', 'sciencevibe.com', 'faithpanda.com', 'neuronation.com', 'dailystormer.name', 'abcbusinessnews.com', 'usanetwork.info', 'bloodandsoil.org', 'blackpigeonspeaks.com', 'conservativebyte.com', 'conservativemedia.com', 'defendevropa.org', 'exposinggovernment.com', 'usafortrumponline.com', 'girlsjustwannahaveguns.com', 'joeforamerica.com', 'mainstreamfakemedia.com', 'patriotcrier.com', 'politicalmayhem.news', 'reformedmedia.net', 'supportisraelnow.com', 'teddystick.com', 'thebostontribune.com', 'thefreepatriot.org', 'truthexam.com', 'acting-man.com', 'almasdarnews.com', 'amityunderground.com', 'councilforthenationalinterest.org', 'cowgernation.com', 'dcbluedot.com', 'factcheckingturkey.com', 'factswanted.com', 'freedomnewsreport.com', 'ihypocrite.net', 'infoscum.com', 'justthefacts.org', 'libertytalk.fm', 'newsammo.com', 'overpassesforamerica.com', 'pmnightlynews.com', 'polipace.com', 'politicalstate.org', 'trumpstudents.org', 'usnationalnews.org', 'libertynews.com', 'republicanssucks.org', 'whitehousewatch.com', 'theblueroute.org', 'private-eye.co.uk', 'constitutionproject.org', 'merica1st.com', 'fmobserver.com', 'the-postillon.com', 'theredshtick.com', 'askingangels.com', 'fashthenation.com', 'thepoliticalbrief.com', 'uslibertywire.club', 'fibrohealthnews.com', '3ccorp.net', 'americansarepissed.com', 'backroombuzz.com', 'bestdailynews.info', 'checkpointasia.net', 'conservativetoday.org', 'patriotreport.com', 'realconservativesunite.com', 'reedcooper.net', 'thelapost.com', 'toptopic.club', 'americadaily.com', 'americanconservativemedia.com', 'breakingwide.com', 'dcalert.com', 'dekalbtimes.com', 'disobedientmedia.com', 'galesburgreporter.com', 'grundyreporter.com', 'hoodsite.com', 'maconreporter.com', 'mcleancountytimes.com', 'metroeastsun.com', 'northegyptnews.com', 'oaklandcitywire.com', 'rawconservativeopinions.com', 'rockfordsun.com', 'sandiegocitywire.com', 'sangamonsun.com', 'seillinoisnews.com', 'southcentralreporter.com', 'swillinoisnews.com', 'silnews.com', 'stfnreport.com', 'conservativebuzz.com', 'westcentralreporter.com', 'albertapressleader.ca', 'americanlibertyreport.com', 'ibleedredwhiteblue.com', 'ca-political.com', 'farleftwatch.com', 'ourfutureinamerica.org', 'libertyheadlines.com', 'mediacircus.com', 'ngo-monitor.org', 'standpointmag.co.uk', 'notiziepericolose.blogspot.it', 'ilmattoquotidiano.it', 'ilfattoquotidaino.it', 'ilquotidaino.wordpress.com', 'liberogiornale.com', 'notiziariosegreto.wordpress.com', 'superbamente.com', 'fonteverificata.it', 'dionidream.com', 'lastella.altervista.org', 'pianetablunews.wordpress.com', 'saltoquantico.org', 'jedanews.it/blog', 'notiziespericolate.com', 'viaggialowcost.altervista.org', 'devinformarti.blogspot.it', 'direttanews24.com', 'tg5stelle.it', 'blogopenyoureyes.altervista.org', 'ilbazarinformazione.blogspot.it', 'notizieinmovimentonews.blogspot.it', 'informaresenzacensure.blogspot.com', 'iocritico.com', 'sostenitori.info', 'irresponsabile.com', 'aidaa-animaliambiente.blogspot.it', 'thuglifevideos.com', 'tgnewsitalia.it/wp', 'livevote.com', 'bigpzone.com', 'wideawakeamerica.com', 'redflagnews.com', 'amusmentic.com', 'automotostar.com', 'educateinspirechange.org/health', 'indiaarising.com', 'uslifetoday.com', 'usuncut.com', 'viraltube.nl', 'yournationnews.com', 'sulatestagiannilannes.blogspot.it', 'tmcrew.org', 'sapereeundovere.com', 'luogocomune.net/lc', 'informatitalia.blogspot.it', 'signoraggio.it', 'albainternazionale.blogspot.it', 'facebook.com/oggi24ore', 'facebook.com/scienza-di-confine-188189217954979', 'facebook.com/stopinvasioneclandestina', 'facebook.com/robyfanpage.it', 'facebook.com/sardegnatoday', 'facebook.com/sadefenza', 'facebook.com/quello-che-non-ci-dicono-107884204213182', 'facebook.com/websocialtv', 'twitter.com/socialtvntw', 'anno3000.altervista.org', 'corrieredelcorsaro.it', 'gazzettadellasera.com', 'gazzettinoweb.altervista.org', 'ilcazzoquotidiano.altervista.org/blog', 'notiziepericolose.blogspot.com', 'facebook.com/ilfalloquotidiano', 'ilgiornaliere.com', 'ilgiomale.it', 'ioco.altervista.org', 'lavocea5stelle.altervista.org', 'liberogiornale.it', 'news24roma.altervista.org', 'news24tg.com', 'news24europa.com', 'notiziea5stelle.com', 'panorana.it', 'sky24ore.it', 'strafattiquotidiani.wordpress.com', 'ilgiornaledeiticinesi.info', 'ticinoinforna.com', 'twitter.com/vigilevittorino', 'awazetribune.com', 'brokenworldnews.com', 'callthecops.net', 'diversitychronicle.wordpress.com', 'fognews.ru', 'freedumjunkshun.com', 'halfwaypost.com', 'holyobserver.com', 'islamicanews.com', 'liberalbias.com', 'lushforlife.com', 'newsbreakers.org', 'newstoad.net', 'objectiveministries.org', 'satirewire.com', 'sportspickle.com', 'stneotscitizen.com', 'thelapine.ca', 'theskunk.org', 'trumpbetrayed.us', 'williambanzai7.blogspot.com', 'wonkie.com', 'amplifyingglass.com', 'duffleblog.com', 'nahadaily.com', 'rockcitytimes.com', 'alternativelyfacts.com', 'alternative-science.com', 'bluenewsnetwork.com', 'bullshitnews.org', 'confederacyofdrones.com', 'eveningharold.com', 'freedomcrossroads.com', 'harddawn.com', 'infobattle.org', 'moronmajority.com', 'npcdaily.com', 'prettycoolsite.com', 'sanctumnews.com', 'schmedium.org', 'stubhillnews.com', 'thatsprettygoodscience.com', 'atlbanana.com', 'thedailymire.com', 'thereisnews.com', 'tittletattle365.com', 'wokennews.com', 'beehivebugle.com', 'bostontribune.com', 'dailyleak.org', 'dandygoat.com', 'freepatriot.org', 'glossynews.com', 'johnnyrobish.com', 'religionlo.com', 'scrappleface.com', 'surrealscoop.com', 'theineptowl.com', 'themadisonmisnomer.com', 'thestatelyharold.com', 'thetimesoftheworld.com', 'topekasnews.com', 'winkprogress.com', 'wundergroundmusic.com', 'collegehumor.com', 'dailycurrant.com', 'empiresports.co', 'freewoodpost.com', 'journaldemourreal.com', 'lightlybraisedturnip.com', 'nationalreport.net', 'thenewsnerd.com', '16wmpo.com', '20minutenews.com', '247newsmedia.com', '24online.news', '24usainfo.com', '24wpn.com', 'abcnews.com.co', 'alabamaobserver.com', 'aldipest.com', 'alertchild.com', 'alynews.com', 'americafans.com', 'americanflare.com', 'americanflavor.news', 'americanpeoplenetwork.com', 'americanpoliticnews.co', 'americanpresident.co', 'americanprides.com', 'americasnewest.com', 'amposts.com', 'anews-24.com', 'anews24.org', 'anonjekloy.tk', 'areyouasleep.com', 'asamericanasapplepie.org', 'associatedmediacoverage.com', 'aurora-news.us', 'badcriminals.com', 'baldwinpost.com', 'baltimoregazette.com', 'battypost.com', 'belgique.actualites.co', 'bients.com', 'bigbluevision.org', 'bignuggetnews.com', 'bizstandardnews.com', 'bluevision.news', 'bluevisionpost.com', 'borderherald.com', 'bostonleader.com', 'breakingtop.world', 'butthatsnoneofmybusiness.com', 'buzzfeedusa.com', 'cartelpress.com', 'cartelreport.com', 'celebtricity.com', 'channel16news.com', 'channel17news.com', 'channel18news.com', 'channel5000.com', 'choiceandtruth.com', 'christiantimesnewspaper.com', 'christiantoday.info', 'chuckcallesto.blogspot.com', 'civictribune.com', 'clancyreport.com', 'clear-politics.com', 'cnn.com.de', 'cnnews3.com', 'consciouslyenlightened.com', 'conservativearmy88.com', 'conservativeflashnews.com', 'conservativeinfocorner.com', 'conservativeinsider.co', 'conservativespirit.com', 'conservativestudio.com', 'consnation.com', 'cooltobeconservative.com', 'daily-sun.com', 'dailyfinesser.com', 'dailyinfobox.com', 'dailyinsidernews.com', 'dailynews10.com', 'dailynews11.com', 'dailynews3.com', 'dailynews33.com', 'dailynews5.com', 'dailynewsposts.info', 'dailypresser.com', 'dailyusaupdate.com', 'damnleaks.com', 'denverinquirer.com', 'departed.co', 'departedmedia.com', 'dineal.com', 'diyhours.net', 'donaldtrumpnews.co', 'donaldtrumppotus45.com', 'drugsofficial.com', 'embols.com', 'endingthefed.com', 'enhlive.com', 'everydaybreakingnews.com', 'everynewshere.com', 'famousviralstories.com', 'fanzinger.com', 'federalistnation.com', 'federalisttribune.com', 'fedsalert.com', 'flashinfo.org', 'flashnewscorner.com', 'floridasunpost.com', 'focusnews.info', 'folksvideo.com', 'fox-news24.com', 'france.actualites.co', 'freddymag.com', 'freedomcrossroads.us', 'freedomsfinalstand.com', 'freshdailyreport.com', 'givemeliberty01.com', 'globalassociatednews.com', 'globalpoliticsnow.com', 'guerilla.news', 'guerillanews.com', 'gummypost.com', 'halturnershow.com', 'headlinebrief.com', 'healthyworldhouse.com', 'heaviermetal.net', 'hotglobalnews.com', 'houstonchronicle-tv.com', 'interestingdailynews.com', 'intrendtoday.com', 'isthatlegit.com', 'jokerviral.com', 'kata33.com', 'klponews.com', 'km8news.com', 'kmt11.com', 'knp7.com', 'konkonsagh.biz', 'krbcnews.com', 'kspm33.com', 'kupr7.com', 'ky12news.com', 'ky6news.com', 'kypo6.com', 'ladylibertysnews.com', 'landrypost.com', 'lastdeplorables.com', 'liberalplug.com', 'liberty-courier.com', 'libertyalliance.com', 'libertywritersnews.com', 'local31news.com', 'localnews33.com', 'lopezreport.com', 'mainerepublicemailalert.com', 'majorthoughts.com', 'maywoodpost.com', 'mbynews.com', 'mckenziepost.com', 'mediazone.news', 'metropolitanworlds.com', 'mississippiherald.com', 'morningnewsusa.com', 'mrnewswatch.com', 'msfanpage.link', 'nationalinsiderpolitics.com', 'nbc.com.co', 'ncscooper.com', 'nephef.com', 'nevo.news', 'newpoliticstoday.com', 'news14kgpn.com', 'news14now.com', 'newsbreakingspipe.com', 'newsbuzzdaily.com', 'newsbysquad.com', 'newsdaily10.com', 'newsdaily12.com', 'newsdaily27.com', 'newsfeedobserver.com', 'newshubs.info', 'newsleak.co', 'newslo.com', 'newsnow17.com', 'newswatch33.com', 'newzmagazine.com', 'noticias365.info', 'openmagazines.com', 'oreillypost.com', 'ourlandofthefree.com', 'paris.actualites.co', 'patriothangout.com', 'persecutes.com', 'politicops.com', 'politicot.com', 'politicsusanews.com', 'powerdaily.us', 'president45donaldtrump.com', 'pressunion.org', 'proudleader.com', 'qualitysharing.com', 'readconservatives.news', 'redcountry.us', 'redinfo.us', 'redrocktribune.com', 'religionmind.com', 'rickwells.us', 'rilenews.com', 'rogue-nation3.com', 'rumorjournal.com', 'satiratribune.com', 'scrapetv.com', 'smhwtfnews.com', 'snoopack.com', 'southernconservativeextra.com', 'spinzon.com', 'statenation.co', 'states-tv.com', 'stgeorgegazette.com', 'straightstoned.com', 'stuppid.com', 'success-street.com', 'supremepatriot.com', 'tdnewswire.com', 'tdtalliance.com', 'telegraphsun.com', 'the-insider.co', 'theavocadonews.com', 'theexaminer.site', 'thefrt.com', 'theinternationalreporter.org', 'theinternetpost.net', 'thelastlineofdefense.org', 'themoralofthestory.us', 'thenationalsun.com', 'thenet24h.com', 'thenochill.com', 'thepoliticaltribune.com', 'thepremiumnews.com', 'theracketreport.com', 'thereporterz.com', 'therightists.com', 'theseattletribune.com', 'thetrumpmedia.com', 'theusa-news.com', 'theusaconservative.com', 'thevalleyreport.com', 'thewashingtonpress.com', 'times.com.mx', 'tmzbreaking.com', 'tmzbusiness.com', 'tmzcomedy.com', 'tmzhiphop.com', 'tmzuncut.com', 'tmzurban.com', 'tmzworldnews.com', 'tmzworldstarnews.com', 'trueamericans.me', 'truetrumpers.com', 'truthkings.com', 'undergroundnewsreport.com', 'unitedmediapublishing.com', 'universepolitics.com', 'usa-conservative.com', 'usa-radio.com', 'usa-television.com', 'usadailyinfo.com', 'usadailypost.us', 'usadailytime.com', 'usadosenews.com', 'usafirstinformation.com', 'usanews4u.us', 'usanewsflash.com', 'usanewshome.com', 'usanewstoday.com', 'usapolitics24hrs.com', 'usapoliticsnow.com', 'usapoliticstoday.com', 'usapoliticszone.com', 'usasnich.com', 'usatodaynews.me', 'usatodaypolitics.com', 'usaworldbox.com', 'ushealthyadvisor.com', 'ushealthylife.com', 'usherald.com', 'usinfonews.com', 'uspoln.com', 'uspostman.com', 'vesselnews.io', 'viralactions.com', 'viralcords.com', 'viraldevil.com', 'viralmugshot.com', 'viralpropaganda.com', 'viralspeech.com', 'viralstuppid.com', 'voxtribune.com', 'washingtonevening.com', 'washingtonfeed.com', 'washingtonpost.com.co', 'wazanews.tk', 'wcpm3.com', 'weconservative.com', 'weekendherald.com', 'wetheproudpatriots.com', 'wftj8news.com', 'whatsupic.com', 'witscience.org', 'wleb21.com', 'wm21news.com', 'wmacnews.com', 'wmb36.com', 'world-politicus.com', 'worldinformation24.info', 'worldnewspolitics.com', 'worldpoliticsnow.com', 'worldpoliticus.com', 'wrejnews.com', 'wrpt16.com', 'wtoe5news.com', 'xbn-news.com', 'zootfeed.com', '24x365live.com', 'a-news24.com', 'americantoday.us', 'antinews.com', 'armyusanews.com', 'benjaminfulford.typepad.com', 'breakingnewsblast.com', 'cnewsgo.com', 'coffeebreakforyou.com', 'conservativepaper.com', 'conservativeview.info', 'dailyfeed.news', 'dailysidnews.com', 'dailythings.world', 'defenseusa.club', 'departedme.com', 'freeinfomedia.com', 'globalrevolutionnetwork.com', 'nativestuff.us', 'newsbreakhere.com', 'newsconservative.com', 'newsformetoday.com', 'newsoftrump.com', 'newsphd.com', 'newsuptoday.com', 'patriotusa.website', 'politicspaper.com', 'rearfront.com', 'redpolitics.us', 'revolutions2040.com', 'teoinfo.com', 'the-global-news.com', 'theworldupdate.com', 'thirdestatenewsgroup.com', 'usa360-tv.com', 'usadailythings24.com', 'uspoliticsinfo.com', 'usaconservativereport.com', 'veteransfordonaldtrump.com', 'weekendpoliticalnews.com', 'worldnewscircle.com', 'world.politics.com', 'mariagrazias.com', 'musicaeanima.com', 'evidenzaliena.mastertopforum.eu', 'ecologiasociale.info', 'frasideilibri.com', 'nonsiamosoli.net', '0585news.com', '24orenotizie.com', '5stellenews.com', 'agenews.eu', 'all-news.tv', 'allnews24.eu', 'attaccomirato.com', 'attivo.news', 'blognews24ore.com', 'borderlinez.com', 'buonamente.com', 'catenaumana.com', 'condividilo.org', 'controinformo.info', 'corrieredisera.it', 'curiosandoonline.com', 'divieto.reattivonews.com', 'essere-informati.it', 'fenixx.org', 'funvideosonline.info', 'gazzetta24.com', 'gazzettanews24.com', 'giornale24ore.com', 'il-giornale.info', 'giornaleilsole.com', 'ilgiornaleitaliano.it', 'ilgiornalenews.com', 'globo365.info', 'ilbellocheavanza.com', 'ilfattodalweb.com', 'ilmeglio-delweb.it', 'ilmondoquotidiano.altervista.org', 'ilmovimentovaavanti.altervista.org', 'ilnotiziario24.com', 'il-quotidiano.info', 'imigliori.info', 'imigliorivideo.co', 'informatitalia.blogspot.com', 'informazionelibera.eu', 'interagisco.net', 'italiani-informati.com', 'iozummo.com', 'italiainmovimento.it', 'italia.imigliori.org', 'italianotizie24.eu', 'italiapatriamia.eu', 'italianosveglia.com', 'italiarialzati.com', 'itruffatori.blogspot.com', 'kevideo.eu', 'larepubbica.it', 'larepubblica.info', 'lastnews24h.com', 'last-webs.com', 'leggimiora.com', 'libero-notizie.com', 'linktoday.it', 'infiltrato.it', 'livewebit.com', 'lolandesevolante.net', 'mattino.ch', 'messaggiero.com', 'morasta.it', 'newscronaca.it', 'newsitalys.com', 'news24roma.com', 'news24italia.it', 'noinvasione.com', 'notixonline.com', 'notixweb.com', 'notizie24h.net', 'notizieflash.eu', 'notizieregionali.com', 'nobufale.it', 'notizieuniversali.it', 'oggi24ore.altervista.org', 'ogginotizie.info', 'perdavvero.com', 'piovegovernoladro.info', 'rainews24.live', 'rassegnastampa.eu', 'reattivonews.com', 'repubblica24.com', 'ribelli.tv', 'riscattonazionale.altervista.org', 'rischiocalcolato.it', 'rivoluzioneoraomaipiu.com', 'robadapazzi.com', 'saper-link-news.com', 'segretodistato.com', 'semplicipensieri.com', 'senzacensura.org', 'siamorimastisoli.com', 'skynew.it', 'skytg24news.com', 'socialbuzz.it', 'socialnotixweb.com', 'soloitaliani.com', 'sputtaniamotutti.reattivonews.org', 'stopeuro.org', 'succedenelmondo.com', 'supernotizie.net', 'thetonicexpress.com', 'tg24italia.com', 'tg-news24.com', 'tg-quotidiano.net', 'tgcom24news.com', 'tutto24.info', 'tuttoinweb.com', 'ultimora24.it', 'videoenotizie.it', 'videoviraliweb.com', 'vocedelweb.com', 'web-news24.com', 'webitalia360.com', 'webtg24.com', 'wowchevideo.com', 'zapping2017.myblog.it', 'zonagrigianews.com', '5ideeinmovimento.blogspot.it', 'attivistam5snews.blogspot.it', 'benitomussolini.altervista.org', 'blogdieles2.altervista.org', 'blogdieles.altervista.org', 'bloginmovimento.altervista.org', 'controinformo.altervista.org', 'corrieredelmattino.altervista.org', 'cvdiariodelpollinho.altervista.org', 'direttainfo.blogspot.it', 'direttanfo.blogspot.it', 'essenzialmentedonna.altervista.org', 'europeannews.altervista.org', 'facebooknotizia.altervista.org', 'ilmessagero.altervista.org', 'ilradicale.altervista.org', 'informazionebycuriosity.altervista.org', 'jedasupport.altervista.org', 'lacucinasiciliana.altervista.org', 'lapillolarossa15.altervista.org', 'newstime24.altervista.org', 'nessuna-censura.com', 'notiziaitalica.altervista.org', 'notiziarioeuropeo.altervista.org', 'politicanews.altervista.org', 'ripuliamolitalia.altervista.org', 'rollingston.altervista.org', 'sadefenza.blogspot.it', 'siamorimastisoli.today', 'tg24h.altervista.org', 'tg24notizie.altervista.org', 'webnotice.altervista.org', 'webnews24h.it', 'cosmofruttariano3m.altervista.org', 'curiosityalive.altervista.org', 'ecocreando.weebly.com', 'guarireoggi.blogspot.it', 'ingannati.it', 'ilrisvegliodellamente.altervista.org', 'paradisefruit.altervista.org', 'thecancermagazine.blogspot.it', 'realscience.altervista.org', 'alleanzadellasalute.info', 'benesserefeed.it', 'comilva.org', 'laleva.org', 'mammainrete.blogspot.com', 'medicinenon.it', 'vacciniinforma.com', 'vivoinsalute.com', 'effervescienza.com', 'filosofiaelogos.it', 'ilnavigatorecurioso.it', 'ilpuntosulmistero.it', 'libreidee.org', 'liberamenteservo.com', 'misteriditalia.it', 'nibiru2012.it', 'ununiverso.it', 'alt-market.com', 'awdnews.com', 'bighairynews.com', 'conservativerefocus.com', 'countdowntozerotime.com', 'counterpsyops.com', 'debunkingskeptics.com', 'dutchsinse.com', 'electionnightgatekeepers.com', 'experimentalvaccines.org', 'farmwars.info', 'gangstergovernment.com', 'govtslaves.info', 'militianews.com', 'patriotrising.com', 'patriotupdate.com', 'platosguns.com', 'politicalcult.com', 'silverbearcafe.com', 'silverstealers.net', 'sonsoflibertyradio.com', 'thelibertymill.com', 'themindunleashed.org', 'themuslimissue.wordpress.com', 'topinfopost.com', 'undergroundworldnews.com', 'wakeupthesheep.com', 'weshapelife.org', '82.221.129.208', 'abeldanger.net', 'americanreviewer.com', 'bullionbullscanada.com', 'conspiracyplanet.com', 'conspiracywire.com', 'disclosuremedia.net', 'ebolahoax.com', 'familysecuritymatters.org', 'fourwinds10.net', 'fprnradio.com', 'freedomforceinternational.com', 'itccs.org', 'jonesreport.com', 'libertymovementradio.com', 'nasamoonhoax.com', 'nationindistress.weebly.com', 'nodisinfo.com', 'oilgeopolitics.net', 'prisonplanet.tv', 'secretsofthefed.com', 'sentinelblog.com', 'silver-coin-investor.com', 'theextinctionprotocol.com', 'truthbroadcastnetwork.com', 'truthfrequencyradio.com', 'wakingupwisconsin.com', 'willyloman.wordpress.com', 'accademiadellaliberta.blogspot.it', 'alcyonpleiadi.blogspot.it', 'altrarealta.blogspot.it', 'appunti2008.blogspot.it', 'compressamente.blogspot.it', 'controinformoblog.altervista.org', 'cortesovrana.it', 'delta-leveritanascoste.blogspot.it', 'freeondarevolution.blogspot.it', 'ilnuovomondodanielereale.blogspot.it', 'ilsapereepotere.blogspot.it', 'jhwhinri.blogspot.it', 'laveritadininconaco.altervista.org', 'madreterra.myblog.it', 'mondonews1.blogspot.it', 'neovitruvian.wordpress.com', 'notiziedalpopolo.altervista.org', 'nwonuovoordinemondiale.wordpress.com', 'olivieromannucci.blogspot.it', 'paoloferrarocdd.blogspot.it', 'presadicoscienza.altervista.org', 'questaitalia.wordpress.com', 'raggioindaco.wordpress.com', 'sitoaurora.wordpress.com', 'tankerenemymeteo.blogspot.it', 'terrarealtime2.blogspot.com', 'veritanwo.altervista.org', 'younetspiegalevele.altervista.org', 'wwwblogdicristian.blogspot.it', 'bluetribune.com', 'freakoutnation.com', 'leftsideofhistory.com', 'resistmedia.org', 'reverbpress.com', 'sourceplanet.net', 'talkleft.com', 'liberalexaminer.com', 'arcmedia.org', 'consamerica.com', 'dailynewscycle.com', 'ih8hillary.com', 'michellemalkin.com', 'mrcblog.com', 'politicalalley.com', 'politicalillusionsexposed.com', 'redbox.news', 'redpeople.us', 'rightvoicemedia.com', 'truthfeed.com', 'uconservative.net', 'usconservative.com', 'qpolitical.com', 'sjlendman.blogspot.com', 'americanborderpatrol.com', 'americantoday.news', 'americaslastlineofdefense.org', 'barnesreview.org', 'bluenationreview.com', 'cnnnext.com', 'creativitymovement.net', 'davidduke.com', 'economicnoise.com', 'factrider.com', 'guccifer2.wordpress.com', 'heightpost.com', 'informetoday.com', 'kagfeed.com', 'macedoniaonline.eu', 'mad-yet.blogspot.com', 'madpatriots.com', 'mminfo24.com', 'newswire-24.com', 'northcrane.com', 'nutritionalanarchy.com', 'pollhype.com', 'realtimepolitics.com', 'rightalert.com', 'rwnofficial.com', 'thegoldandoilguy.com', 'urdoca.com', 'usdefensewatch.com', 'aanirfan.blogspot.co.uk', 'anonnews.co', 'automaticearth.com', 'boilingfrogspost.com', 'bvanews.com', 'conservativeoutfitters.com', 'morningledger.com', 'newcenturytimes.com', 'politicsintheusa.com', 'powerpoliticians.com', 'theantimedia.org', 'thegoldandoilguy.com/articles', 'usviewer.com', 'veteransnewsnow.com', 'abriluno.com', 'allnews4us.com', 'alternativemediasyndicate.net', 'bigamericannews.com', 'cannasos.com', 'christwire.org', 'deadlyclear.wordpress.com', 'immediatesafety.org', 'itaglive.com', 'jewsnews.co.il', 'liberaldarkness.com', 'libertyblitzkrieg.com', 'libertywriters.com', 'mentor2day.com', 'myfreshnews.com', 'newsfrompolitics.com', 'newslogue.com', 'newsmutiny.com', 'newswatch28.com', 'noticias-frescas.com', 'notizzia.com', 'politicalsitenews.com', 'reagancoalition.com', 'reporter.bz', 'smag31.com', 'subjectpolitics.com', 'thebigriddle.com', 'thetruthdivision.com', 'usamagazinestudio.com', 'viralliberty.com', 'viralnewssocial.com', '24newsflash.com', '70news.wordpress.com', 'abcnewsgo.co', 'alternativemediasyndicate.com', 'americanoverlook.com', 'blackinusrancenews.com', 'cbsnews.com.co', 'channel-7-news.com', 'conservativeinfidel.com', 'ddemocraticunderground.com', 'drudgereport.com.co', 'empirehearland.com', 'en-vovle.com', 'enabon.com', 'enduringvision.com', 'eutopia.buzz', 'givemeliberity01.com', 'goneleft.com', 'gopthedailydose.com', 'healthycareandbeauty.com', 'itmakessenseblog.com', 'iwanttoexplore.com', 'ladylibertynews.com', 'learnprogress.org', 'maganews.co', 'megynkelly.us', 'mpidailymagazine.com', 'mrconservative.com', 'myzonetoday.com', 'nationonenews.com', 'newsbbc.net', 'newsmagazine.com', 'onlineconservativepress.com', 'politicalo.com', 'politicalupdator.com', 'politicono.com', 'proamericanews.com', 'proudcons.com', 'rhotv.com', 'thatviralfeed.net', 'theamericanindependent.wordpress.com', 'thecontroversialfiles.net', 'thelastgreatstand.com', 'trumpvision365.com', 'usa2016elections.com', 'usadailypolitics.com', 'usainfobox.com', 'usanewsinsider.com', 'usanewspolitics.com', 'usaonlinepolitics.com', 'usaphase.com', 'usatoday.com.co', 'usatwentyfour.com', 'usconservativetoday.com', 'usfanzone.com', 'uspoliticslive.com', 'washingtonfed.com', 'worldpoliticsus.com', 'worldrumor.com', 'worldstoriestoday.com', 'yourfunpage.com', 'twitter.com/capnews', 'creambmp.com', 'derfmagazine.com', 'duhprogressive.com', 'empirenews.com', 'cc.com/indecision', 'news-hound.com', 'politicalears.com', 'sprotspickle.com', 'theuspatriot.com', 'ageofshitlords.com', 'americamaxnews.com', 'en.bbarta24.net', 'facebook.com/conservativepoliticstoday', 'thedcgazette.com', 'hardnews.network', 'en.hawarnews.com', 'knightstemplarinternational.com', 'youtube.com/nextnewsnetwork', 'patriotbeat.com', 'prideof-america.org', 'progresstribune.com', 'rightalerts.com', 'shafaaq.com', 'shariawatch.org.uk', 'speisa.com', 'studionewsnetwork.com', 'mynewsguru.com', 'thepeoplestruth.com', 'facebook.com/thewashingtonstandard', 'usright-left.com', 'westernsentinel.com', 'maga2020.us', 'ff911truthandunity.org', 'hellochristian.com', 'homeopathyjournal.net', 'jeffereyjaxen.com', 'sp.neuronation.com/en', 'online-updates.net', 'sitsshow.blogspot.com', 'cnn-business-news.ga', 'kbc14.com', 'kcst7.com', 'kf13.com', 'krb7.com', 'kty24news.com', 'nydailynews-tv.com', 'observeronline.news', 'themiamigazette.com', 'werk35.com', 'westfieldpost.com', 'wrpm33.com', 'wy21news.com', 'freedomjunkshun.com', 'nunadisbereel.com', 'politicass.com', 'therealshtick.com', 'bestthings.us', 'blackinsurancenews.com', 'bluelinestrong.net', 'conservative7.com', 'conservativepoliticus.com', 'ilovenativeamericans.us', 'mediamaxzone.com', 'nativeamericans.us', 'positivedaily.com', 'reflectionofmind.org', 'thesolexchange.com', 'topinfopost.con', 'urbanimagemagazine.com', 'welovenative.com', 'americankabuki.blogspot.com', 'checkoutthehealthyworld.com', 'collectivelyconscious.net', 'galacticconnection.com', 'lifeprevention.com', 'rawforbeauty.com', 'realplanetnews.com', 'worldwidehealthy.com', 'magafeed.com', 'megafreshnews.com', 'nmws.us', 'dailystormer.su', 'borderlandalternativemedia.com', 'politicslive.net', 'usadailyreview.com', 'amg-news.com', 'medicalcensorship.com', 'gettr.com', 'youtube.com', 'facebook.com', 'jw.org', 'shafaq.com', 'cureus.com', 'youm7.com', 'informer.rs', 'thieme.de', 'en.granma.cu', 'weibo.com', 'wellnessachiever.net', 'fr24news.com', 'rightsmarts.com', 'thedailycheck.net', 'populartechnology.net', 'presidentialhill.com', 'propornot.com', 'gmowatch.com', 'newsglobal.com', 'muricatoday.com', 'medusamagazine.com', 'extraordinaryinfo.com', 'vancouvertimes.org', 'andersoninstitute.com', 'bostonbroadside.com', 'conservativeglobe.com', 'dissident-mag.com', 'gript.ie', 'headlineusa.com', 'israel365news.com', 'livingstonetoday.com', 'mena.org.eg/en', 'morningpoll.com', 'nationalzero.com', 'patriotuproar.com', 'springfieldcitywire.com', 'steadfastclash.com', 'thegoptimes.com', 'thejewishvoice.com', 'thelibertyrevolution.com', 'usacrime.com/hoodsite', 'wndnewscenter.org', 'yournews.com']

In [16]:
print(len(domains))

4271


In [41]:
domains_str=",".join(domains[:500])

In [30]:
import json

In [31]:
domains_dict = {"d": domains_str}

In [32]:
json_domains=json.dumps(domains_dict)

In [33]:
json_domains

'{"d": "dailymail.co.uk,rumble.com,rawstory.com,indiatimes.com,breitbart.com,theepochtimes.com,thegatewaypundit.com,zerohedge.com,cbn.com,rt.com,left.it,newsmax.com,news18.com,gnews.org,newsbreak.com,economictimes.indiatimes.com,thefederalist.com,dailywire.com,dailycaller.com,washingtontimes.com,stream.org,childrenshealthdefense.org,chosun.com,tichyseinblick.de,lifenews.com,express.co.uk,dailykos.com,townhall.com,lifesitenews.com,theblaze.com,valeursactuelles.com,telesurtv.net,occupydemocrats.com,jihadwatch.org,esrt.press,theonion.com,expose-news.com,lemediaen442.fr,redstate.com,palmerreport.com,politicususa.com,bitchute.com,lecourrier-du-soir.com,elamerican.com,rsbnetwork.com,alternet.org,health.news,amgreatness.com,judicialwatch.org,firstpost.com,peta.org,politicalflare.com,westernjournal.com,thecountersignal.com,bvoltaire.fr,greenpeace.org,wsws.org,jezebel.com,hannity.com,francesoir.fr,pagesix.com,twitchy.com,skynews.com.au,laverita.info,ria.ru,babylonbee.com,crooksandliars.com,brea

In [1]:
all_articles = newsapi.get_everything(
    q='climate change OR global warming OR climate crisis OR sustainability OR renewable energy',
            domains=domains_str,
                                      from_param='2023-10-05',
                                      to='2023-10-07',
                                      language='en',
                                      sort_by='popularity',)

NameError: name 'newsapi' is not defined

In [5]:
all_articles["articles"]

NameError: name 'all_articles' is not defined

In [44]:
domains[:5]

['dailymail.co.uk',
 'rumble.com',
 'rawstory.com',
 'indiatimes.com',
 'breitbart.com']

In [45]:
domains_set = set(domains)

In [54]:
'Theonion.com'.lower() in domains_str

True

In [55]:
from urllib.parse import urlparse


In [1]:
import tldextract

In [4]:
tldextract.extract('https://www.mindbodygreen.com/articles/3-skin-stacking-hacks-for-firmer-smoother-complexion').suffix

'com'

In [61]:
urlparse('https://www.mindbodygreen.com/articles/3-skin-stacking-hacks-for-firmer-smoother-complexion')

ParseResult(scheme='https', netloc='www.mindbodygreen.com', path='/articles/3-skin-stacking-hacks-for-firmer-smoother-complexion', params='', query='', fragment='')